# 지하철역 소재와 서울시 구/행정동 연결

### 1. 데이터 준비

In [1]:
# 1-1. 필요 라이브러리 불러오기

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

# 한글 폰트 설정
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

In [2]:
# 1-2. 데이터 파일 불러오기
# 행정동 경계파일 불러오기
a_dong_gdf = gpd.read_file\
    ('../4.Public_transportation/BND_ADM_DONG_PG_Adong/BND_ADM_DONG_PG.shp',
     encoding='euc-kr')

# 법정동 경계파일 불러오기
l_dong_gdf = gpd.read_file\
    ('../4.Public_transportation/LSMD_ADM_SECT_UMD_Ldong/LSMD_ADM_SECT_UMD_11_202409.shp',
     encoding='cp949')

# 구 경계파일 불러오기
gu_gdf = gpd.read_file\
    ('../4.Public_transportation/LARD_ADM_SECT_SGG_gu/LARD_ADM_SECT_SGG_11_202405.shp',
     encoding='cp949')

# 구별 행정동 목록파일 불러오기
gu_a_dong_df = pd.read_csv\
    ('../4.Public_transportation/Gu_and_Administrative_dong.csv', encoding='cp949')

# 구별 법정동 목록파일 불러오기
gu_l_dong_df = pd.read_csv\
    ('../4.Public_transportation/Gu_and_Legal_dong.csv', encoding='cp949')

# 아파트 주소 데이터 불러오기 (2019~2023)
apt_l_and_l_df = pd.read_csv\
    ('Data_preprocessing/combined_geo.csv')

# 아파트 주소 데이터 불러오기 (~2018.09)
apt_l_and_l_201809 = pd.read_csv\
    ('201809_address.csv')

# 아파트 주소 데이터 불러오기 (~2018.12)
apt_l_and_l_201812 = pd.read_csv\
    ('201812_address.csv')

### 2. 불러온 데이터 확인

In [3]:
print("=== 데이터 현황 ===")
print("\n[구 경계 데이터]")
print("- 데이터 크기:", gu_gdf.shape)
print("- 컬럼 목록:", gu_gdf.columns.tolist())
print("- 구 목록:", sorted(gu_gdf['SGG_NM'].tolist()))
print("- 좌표계:", gu_gdf.crs)

print("\n[행정동 경계 데이터]")
print("- 데이터 크기:", a_dong_gdf.shape)
print("- 컬럼 목록:", a_dong_gdf.columns.tolist())
print("- 좌표계:", a_dong_gdf.crs)

print("\n[법정동 경계 데이터]")
print("- 데이터 크기:", l_dong_gdf.shape)
print("- 컬럼 목록:", l_dong_gdf.columns.tolist())
print("- 좌표계:", l_dong_gdf.crs)

print("\n[아파트 2019~2023 위도 경도 데이터]")
print("- 데이터 크기:", apt_l_and_l_df.shape)
print("- 컬럼 목록:", apt_l_and_l_df.columns.tolist())

print("\n[아파트 ~2018.09 위도 경도 데이터]")
print("- 데이터 크기:", apt_l_and_l_201809.shape)
print("- 컬럼 목록:", apt_l_and_l_201809.columns.tolist())

print("\n[아파트 ~2018.12 위도 경도 데이터]")
print("- 데이터 크기:", apt_l_and_l_201812.shape)
print("- 컬럼 목록:", apt_l_and_l_201812.columns.tolist())


=== 데이터 현황 ===

[구 경계 데이터]
- 데이터 크기: (25, 5)
- 컬럼 목록: ['ADM_SECT_C', 'SGG_NM', 'SGG_OID', 'COL_ADM_SE', 'geometry']
- 구 목록: ['서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구', '서울특별시 관악구', '서울특별시 광진구', '서울특별시 구로구', '서울특별시 금천구', '서울특별시 노원구', '서울특별시 도봉구', '서울특별시 동대문구', '서울특별시 동작구', '서울특별시 마포구', '서울특별시 서대문구', '서울특별시 서초구', '서울특별시 성동구', '서울특별시 성북구', '서울특별시 송파구', '서울특별시 양천구', '서울특별시 영등포구', '서울특별시 용산구', '서울특별시 은평구', '서울특별시 종로구', '서울특별시 중구', '서울특별시 중랑구']
- 좌표계: PROJCS["Korea_2000_Korea_Central_Belt_2010",GEOGCS["GCS_Korea_2000",DATUM["Korean_Geodetic_Datum_2002",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6737"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",38],PARAMETER["central_meridian",127],PARAMETER["scale_factor",1],PARAMETER["false_easting",200000],PARAMETER["false_northing",600000],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]



In [4]:
apt_2018_07 = pd.concat([apt_l_and_l_201812, apt_l_and_l_201809], ignore_index=True)
apt_2018_07["NO"] = '2018_' + apt_2018_07['NO'].astype('str').str.zfill(5)

selected_apt_2018_07 = apt_2018_07[['NO', 'latitude', 'longitude', '주소_합치기']]
selected_apt_2018_07.columns = ['NO', 'latitude', 'longitude', 'address']
for_dong_apt = pd.concat([selected_apt_2018_07, apt_l_and_l_df], ignore_index=True)
for_dong_apt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287046 entries, 0 to 287045
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   NO         287046 non-null  object 
 1   latitude   287046 non-null  float64
 2   longitude  287046 non-null  float64
 3   address    287046 non-null  object 
dtypes: float64(2), object(2)
memory usage: 8.8+ MB


### 3. 데이터 전처리

In [5]:
# 3.1 좌표계 통일 (모두 EPSG:4326으로 변환)
a_dong_gdf = a_dong_gdf.to_crs("EPSG:4326")
l_dong_gdf = l_dong_gdf.to_crs("EPSG:4326")
gu_gdf = gu_gdf.to_crs("EPSG:4326")

# 3.2 아파트 데이터를 GeoDataFrame으로 변환
apt_geometry = [Point(xy) for xy in zip(for_dong_apt['longitude'], for_dong_apt['latitude'])]
apt_gdf = gpd.GeoDataFrame(
    for_dong_apt, 
    geometry=apt_geometry,
    crs="EPSG:4326"
)

print("\n=== 전처리 결과 ===")
print("- 행정동 좌표계:", a_dong_gdf.crs)
print("- 법정동 좌표계:", l_dong_gdf.crs)
print("- 구 좌표계:", gu_gdf.crs)
print("- 아파트 좌표계:", apt_gdf.crs)


=== 전처리 결과 ===
- 행정동 좌표계: EPSG:4326
- 법정동 좌표계: EPSG:4326
- 구 좌표계: EPSG:4326
- 아파트 좌표계: EPSG:4326


### 4. 공간 분석 작업

In [6]:
# 4.1 아파트와 구 매칭
apt_with_gu = gpd.sjoin(apt_gdf, gu_gdf, how='left', predicate='within')

# 4.2 아파트와 행정동 매칭
apt_with_a_dong = gpd.sjoin(apt_gdf, a_dong_gdf, how='left', predicate='within')

# 4.2 아파트와 법정동 매칭
apt_with_l_dong = gpd.sjoin(apt_gdf, l_dong_gdf, how='left', predicate='within')

# 4.3 최종 데이터프레임 생성
result_df = pd.DataFrame({
    'NO': apt_with_gu['NO'],
    'address': apt_with_gu['address'],
    'gu': apt_with_gu['SGG_NM'],
    'a_dong': apt_with_a_dong['ADM_NM'],
    'l_dong': apt_with_l_dong['EMD_NM']
})

# # 4.4 서울시 외 지역 제거
# result_df = result_df.dropna(subset=['gu', 'dong'])
# result_df = result_df.sort_values(['gu', 'dong', 'line', 'station_name'])
# result_df.info()

# 4.5 분석 결과 출력
# print(f"서울시 지하철역 수: {len(result_df)}개")

# gu_station_count = result_df.groupby('gu').size().sort_values(ascending=False)
# print("\n구별 지하철역 수:")
# print(gu_station_count)
# result_df.head()
# result_df['gu'].nunique(), result_df['dong'].nunique()

In [7]:
result_df.isna().sum()
result_df.head(), result_df.tail()

(           NO                address         gu a_dong l_dong
 0  2018_00001     서울특별시 중랑구  면목로 429  서울특별시 중랑구   면목본동    면목동
 1  2018_00002  서울특별시 성북구  낙산길 243-15  서울특별시 성북구    삼선동  삼선동2가
 2  2018_00003  서울특별시 중랑구  봉화산로48길 62  서울특별시 중랑구   상봉1동    상봉동
 3  2018_00004     서울특별시 중랑구  면목로 429  서울특별시 중랑구   면목본동    면목동
 4  2018_00005    서울특별시 중랑구  용마산로 252  서울특별시 중랑구   면목7동    면목동,
                 NO              address         gu a_dong l_dong
 287041  2023_35638  서울특별시 은평구 연서로28길 12  서울특별시 은평구    대조동    대조동
 287042  2023_35639    서울특별시 은평구 진흥로 153  서울특별시 은평구    대조동    대조동
 287043  2023_35640  서울특별시 강남구 광평로19길 15  서울특별시 강남구   일원본동    일원동
 287044  2023_35641  서울특별시 강남구 광평로19길 15  서울특별시 강남구   일원본동    일원동
 287045  2023_35642   서울특별시 서초구 반포대로 275  서울특별시 서초구   반포2동    반포동)

In [8]:
result_df.to_csv('Data_Preprocessing/address_with_a_dong_and_l_dong.csv', index=False, encoding='utf-8')

In [9]:
result_df.to_csv('Data_Preprocessing/address_with_a_dong_and_l_dong_plus_2018.csv', index=False, encoding='utf-8')
combined_origin = pd.read_csv('Data_Preprocessing/combined_origin.csv')

In [10]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 287046 entries, 0 to 287045
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   NO       287046 non-null  object
 1   address  287046 non-null  object
 2   gu       287046 non-null  object
 3   a_dong   287046 non-null  object
 4   l_dong   287046 non-null  object
dtypes: object(5)
memory usage: 13.1+ MB


In [11]:
combined_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 20 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   NO        251096 non-null  object 
 1   시군구       251096 non-null  object 
 2   번지        251096 non-null  object 
 3   본번        251096 non-null  int64  
 4   부번        251096 non-null  int64  
 5   단지명       251096 non-null  object 
 6   전용면적(㎡)   251096 non-null  float64
 7   계약년월      251096 non-null  int64  
 8   계약일       251096 non-null  int64  
 9   거래금액(만원)  251096 non-null  int64  
 10  동         251096 non-null  object 
 11  층         251096 non-null  int64  
 12  매수자       251096 non-null  object 
 13  매도자       251096 non-null  object 
 14  건축년도      251096 non-null  int64  
 15  도로명       251096 non-null  object 
 16  해제사유발생일   251096 non-null  object 
 17  거래유형      251096 non-null  object 
 18  중개사소재지    251096 non-null  object 
 19  등기일자      251096 non-null  object 
dtypes: f

In [12]:
ori_apt_2018_07 = apt_2018_07[['NO', '시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', 
                               '계약년월', '계약일', '거래금액(만원)', '동', '층', '매수자', '매도자', 
                               '건축년도', '도로명', '해제사유발생일', '거래유형', '중개사소재지', 
                               '등기일자']]
combined_df = pd.concat([ori_apt_2018_07, combined_origin], ignore_index=True)
combined_df = combined_df[['NO', '계약년월', '계약일']]
combined_df = combined_df.astype('str')
combined_df['contract_date'] = combined_df['계약년월'] + combined_df['계약일']
combined_df = combined_df[['NO', 'contract_date']]
# combined_df['contract_date'] = combined_df['contract_date'].astype('str')
combined_df['contract_date'] = pd.to_datetime(combined_df['contract_date'])
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287046 entries, 0 to 287045
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   NO             287046 non-null  object        
 1   contract_date  287046 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 4.4+ MB


In [13]:
result_df['gu'] = result_df['gu'].str.replace('서울특별시 ', '', regex=False)

result_df['gu_adong'] = result_df['gu'] + '_' + result_df['a_dong']
result_df['gu_ldong'] = result_df['gu'] + '_' + result_df['l_dong']
result_df.info()
result_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 287046 entries, 0 to 287045
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   NO        287046 non-null  object
 1   address   287046 non-null  object
 2   gu        287046 non-null  object
 3   a_dong    287046 non-null  object
 4   l_dong    287046 non-null  object
 5   gu_adong  287046 non-null  object
 6   gu_ldong  287046 non-null  object
dtypes: object(7)
memory usage: 17.5+ MB


,NO,address,gu,a_dong,l_dong,gu_adong,gu_ldong
0,2018_00001,서울특별시 중랑구 면목로 429,중랑구,면목본동,면목동,중랑구_면목본동,중랑구_면목동
1,2018_00002,서울특별시 성북구 낙산길 243-15,성북구,삼선동,삼선동2가,성북구_삼선동,성북구_삼선동2가
2,2018_00003,서울특별시 중랑구 봉화산로48길 62,중랑구,상봉1동,상봉동,중랑구_상봉1동,중랑구_상봉동
3,2018_00004,서울특별시 중랑구 면목로 429,중랑구,면목본동,면목동,중랑구_면목본동,중랑구_면목동
4,2018_00005,서울특별시 중랑구 용마산로 252,중랑구,면목7동,면목동,중랑구_면목7동,중랑구_면목동


In [14]:
apt_dong = pd.merge(result_df, combined_df, how='inner', on='NO')

In [15]:
apt_dong = apt_dong[['NO', 'contract_date', 'address', 'gu', 'a_dong', 'l_dong', 'gu_adong', 'gu_ldong']]
apt_dong

,NO,contract_date,address,gu,a_dong,l_dong,gu_adong,gu_ldong
0,2018_00001,2018-12-31,서울특별시 중랑구 면목로 429,중랑구,면목본동,면목동,중랑구_면목본동,중랑구_면목동
1,2018_00002,2018-12-31,서울특별시 성북구 낙산길 243-15,성북구,삼선동,삼선동2가,성북구_삼선동,성북구_삼선동2가
2,2018_00003,2018-12-31,서울특별시 중랑구 봉화산로48길 62,중랑구,상봉1동,상봉동,중랑구_상봉1동,중랑구_상봉동
3,2018_00004,2018-12-31,서울특별시 중랑구 면목로 429,중랑구,면목본동,면목동,중랑구_면목본동,중랑구_면목동
4,2018_00005,2018-12-31,서울특별시 중랑구 용마산로 252,중랑구,면목7동,면목동,중랑구_면목7동,중랑구_면목동
...,...,...,...,...,...,...,...,...
287041,2023_35638,2023-01-02,서울특별시 은평구 연서로28길 12,은평구,대조동,대조동,은평구_대조동,은평구_대조동
287042,2023_35639,2023-01-02,서울특별시 은평구 진흥로 153,은평구,대조동,대조동,은평구_대조동,은평구_대조동
287043,2023_35640,2023-01-01,서울특별시 강남구 광평로19길 15,강남구,일원본동,일원동,강남구_일원본동,강남구_일원동
287044,2023_35641,2023-01-01,서울특별시 강남구 광평로19길 15,강남구,일원본동,일원동,강남구_일원본동,강남구_일원동


In [16]:
# 데이터 준비
apt_dong['contract_date'] = pd.to_datetime(apt_dong['contract_date'])
apt_dong['contract_ym'] = apt_dong['contract_date'].dt.to_period('M')  # 계약 년월 추가
apt_dong['count'] = 1  # 거래 건수 세기용 컬럼

# 1. 계약 년월별 행정동, 법정동 거래 건수 집계
monthly_counts = apt_dong.groupby(['contract_ym', 'gu_adong', 'gu_ldong'])['count'].sum().reset_index()

# 2. 거래 물량 계산 함수
def calculate_lag_counts(df, group_col, value_col):
    # 그룹화 및 이동 평균 계산
    grouped = df.groupby(group_col)
    
    # 직전 1개월
    df['prev_1_month'] = grouped[value_col].shift(1)
    
    # 직전 3개월 합계
    df['prev_3_months'] = (
        grouped[value_col]
        .rolling(window=3, min_periods=1)
        .sum()
        .shift(1)
        .reset_index(level=group_col, drop=True)
    )
    
    # 직전 6개월 합계
    df['prev_6_months'] = (
        grouped[value_col]
        .rolling(window=6, min_periods=1)
        .sum()
        .shift(1)
        .reset_index(level=group_col, drop=True)
    )
    
    return df

# 3. 행정동 기준 거래 물량 계산
adong_counts = calculate_lag_counts(monthly_counts, group_col='gu_adong', value_col='count')
adong_counts = adong_counts.rename(columns={
    'prev_1_month': 'adong_prev_1_month',
    'prev_3_months': 'adong_prev_3_months',
    'prev_6_months': 'adong_prev_6_months'
})

# 4. 법정동 기준 거래 물량 계산
ldong_counts = calculate_lag_counts(monthly_counts, group_col='gu_ldong', value_col='count')
ldong_counts = ldong_counts.rename(columns={
    'prev_1_month': 'ldong_prev_1_month',
    'prev_3_months': 'ldong_prev_3_months',
    'prev_6_months': 'ldong_prev_6_months'
})

# 5. 두 결과를 결합
result = pd.merge(adong_counts, ldong_counts, on=['contract_ym', 'gu_adong', 'gu_ldong', 'count'], how='left')

# 6. 원본 데이터와 결합하여 최종 결과 생성
final_result = pd.merge(apt_dong, result, on=['contract_ym', 'gu_adong', 'gu_ldong'], how='left')

In [17]:
# 결과 확인
print(final_result.head())

           NO contract_date                address   gu a_dong l_dong  \
0  2018_00001    2018-12-31     서울특별시 중랑구  면목로 429  중랑구   면목본동    면목동   
1  2018_00002    2018-12-31  서울특별시 성북구  낙산길 243-15  성북구    삼선동  삼선동2가   
2  2018_00003    2018-12-31  서울특별시 중랑구  봉화산로48길 62  중랑구   상봉1동    상봉동   
3  2018_00004    2018-12-31     서울특별시 중랑구  면목로 429  중랑구   면목본동    면목동   
4  2018_00005    2018-12-31    서울특별시 중랑구  용마산로 252  중랑구   면목7동    면목동   

   gu_adong   gu_ldong contract_ym  count_x  count_y  adong_prev_1_month  \
0  중랑구_면목본동    중랑구_면목동     2018-12        1        6                 3.0   
1   성북구_삼선동  성북구_삼선동2가     2018-12        1        1                 1.0   
2  중랑구_상봉1동    중랑구_상봉동     2018-12        1        3                 5.0   
3  중랑구_면목본동    중랑구_면목동     2018-12        1        6                 3.0   
4  중랑구_면목7동    중랑구_면목동     2018-12        1        4                 2.0   

   adong_prev_3_months  adong_prev_6_months  ldong_prev_1_month  \
0                  9.0               

In [18]:
final_result = final_result[['NO', 'contract_date', 'contract_ym', 
                             'gu', 'a_dong', 'l_dong', 'gu_adong', 'gu_ldong', 
                             'adong_prev_1_month', 'adong_prev_3_months', 'adong_prev_6_months', 
                             'ldong_prev_1_month', 'ldong_prev_3_months', 'ldong_prev_6_months']]

In [19]:
final_result['adong_prev_6_months'].max()

617.0

In [20]:
# 8번 컬럼부터 마지막 컬럼까지 선택
cols_to_convert = final_result.columns[8:]

# NaN 값을 0으로 대체하고 int16으로 변환
final_result[cols_to_convert] = final_result[cols_to_convert].fillna(0).astype('int16')

# 결과 확인
print(final_result.dtypes)


NO                             object
contract_date          datetime64[ns]
contract_ym                 period[M]
gu                             object
a_dong                         object
l_dong                         object
gu_adong                       object
gu_ldong                       object
adong_prev_1_month              int16
adong_prev_3_months             int16
adong_prev_6_months             int16
ldong_prev_1_month              int16
ldong_prev_3_months             int16
ldong_prev_6_months             int16
dtype: object


In [21]:
final_result.to_csv('apt_with_adong_and_ldong_calc_prev_1_3_6.csv', index=False, encoding='utf-8')

In [22]:
E_df = pd.read_csv('forDP/from_Jun_주요국 통화의 대원화환율.csv', encoding='cp949')

In [23]:
E_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   날짜             1238 non-null   object 
 1   원/미국달러(매매기준율)  1238 non-null   object 
 2   원/위안(매매기준율)    1238 non-null   float64
 3   원/일본엔(100엔)    1238 non-null   object 
dtypes: float64(1), object(3)
memory usage: 38.8+ KB


In [24]:
E_df.columns = ['date', 'USD_exchange_rate', 'CNY_exchange_rate', 'JPY_exchange_rate']
E_df['date'] = pd.to_datetime(E_df['date'])
E_df.info(), E_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1238 entries, 0 to 1237
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               1238 non-null   datetime64[ns]
 1   USD_exchange_rate  1238 non-null   object        
 2   CNY_exchange_rate  1238 non-null   float64       
 3   JPY_exchange_rate  1238 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 38.8+ KB


(None,
         date USD_exchange_rate  CNY_exchange_rate JPY_exchange_rate
 0 2019-01-02          1,118.10             162.76          1,019.47
 1 2019-01-03          1,119.00             163.02          1,044.87
 2 2019-01-04          1,127.50             163.74          1,046.36
 3 2019-01-07          1,125.20             163.72          1,038.34
 4 2019-01-08          1,118.00             163.21          1,028.85)

In [25]:
E_df["USD_exchange_rate"] = \
        E_df["USD_exchange_rate"].map(lambda v : float(str(v).replace(",", "")))

E_df["JPY_exchange_rate"] = \
        E_df["JPY_exchange_rate"].map(lambda v : float(str(v).replace(",", "")))

In [26]:
apt_dong_transaction_count_and_E = pd.merge(final_result, E_df, how='outer', left_on='contract_date', right_on='date')
apt_dong_transaction_count_and_E = \
      apt_dong_transaction_count_and_E[['NO', 'contract_date', 'contract_ym', 'date', 'USD_exchange_rate', 
       'CNY_exchange_rate', 'JPY_exchange_rate', 'gu', 'a_dong', 'l_dong',
       'gu_adong', 'gu_ldong', 'adong_prev_1_month', 'adong_prev_3_months',
       'adong_prev_6_months', 'ldong_prev_1_month', 'ldong_prev_3_months',
       'ldong_prev_6_months']]
apt_dong_transaction_count_and_E.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287046 entries, 0 to 287045
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   NO                   287046 non-null  object        
 1   contract_date        287046 non-null  datetime64[ns]
 2   contract_ym          287046 non-null  period[M]     
 3   date                 173507 non-null  datetime64[ns]
 4   USD_exchange_rate    173507 non-null  float64       
 5   CNY_exchange_rate    173507 non-null  float64       
 6   JPY_exchange_rate    173507 non-null  float64       
 7   gu                   287046 non-null  object        
 8   a_dong               287046 non-null  object        
 9   l_dong               287046 non-null  object        
 10  gu_adong             287046 non-null  object        
 11  gu_ldong             287046 non-null  object        
 12  adong_prev_1_month   287046 non-null  int16         
 13  adong_prev_3_m

In [27]:
# 환율 관련 컬럼에 null이 있는 행들만 선택
null_df = apt_dong_transaction_count_and_E[apt_dong_transaction_count_and_E[['USD_exchange_rate', 'CNY_exchange_rate', 'JPY_exchange_rate']].isnull().any(axis=1)]

In [28]:
null_df['contract_date'].unique()

<DatetimeArray>
['2018-07-01 00:00:00', '2018-07-02 00:00:00', '2018-07-03 00:00:00',
 '2018-07-04 00:00:00', '2018-07-05 00:00:00', '2018-07-06 00:00:00',
 '2018-07-07 00:00:00', '2018-07-08 00:00:00', '2018-07-09 00:00:00',
 '2018-07-10 00:00:00',
 ...
 '2023-12-03 00:00:00', '2023-12-09 00:00:00', '2023-12-10 00:00:00',
 '2023-12-16 00:00:00', '2023-12-17 00:00:00', '2023-12-23 00:00:00',
 '2023-12-24 00:00:00', '2023-12-25 00:00:00', '2023-12-30 00:00:00',
 '2023-12-31 00:00:00']
Length: 771, dtype: datetime64[ns]

In [29]:
# 1. 'date' 컬럼을 기준으로 데이터프레임을 정렬 (날짜 순)
apt_dong_transaction_count_and_E.sort_values(by='date', inplace=True)

# 2. 'ffill'을 사용하여 환율 결측치 채우기 (환율 컬럼만 선택)
rate_columns = ['USD_exchange_rate', 'CNY_exchange_rate', 'JPY_exchange_rate']
apt_dong_transaction_count_and_E[rate_columns] = apt_dong_transaction_count_and_E[rate_columns].fillna(method='ffill', axis=0)

# 3. 추가적으로 빈 날짜가 있을 경우, 이전 값으로 채우기
# 'limit'을 설정해 원하는 만큼 전날, 전전날 등으로 채우기
apt_dong_transaction_count_and_E[rate_columns] = apt_dong_transaction_count_and_E[rate_columns].fillna(method='ffill', limit=2, axis=0)

# 4. 결과 확인 (결측치가 남아있는지 확인)
print(apt_dong_transaction_count_and_E.isna().sum())

NO                          0
contract_date               0
contract_ym                 0
date                   113539
USD_exchange_rate           0
CNY_exchange_rate           0
JPY_exchange_rate           0
gu                          0
a_dong                      0
l_dong                      0
gu_adong                    0
gu_ldong                    0
adong_prev_1_month          0
adong_prev_3_months         0
adong_prev_6_months         0
ldong_prev_1_month          0
ldong_prev_3_months         0
ldong_prev_6_months         0
dtype: int64


C:\Users\human\AppData\Local\Temp\ipykernel_5092\352665172.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  apt_dong_transaction_count_and_E[rate_columns] = apt_dong_transaction_count_and_E[rate_columns].fillna(method='ffill', axis=0)
C:\Users\human\AppData\Local\Temp\ipykernel_5092\352665172.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  apt_dong_transaction_count_and_E[rate_columns] = apt_dong_transaction_count_and_E[rate_columns].fillna(method='ffill', limit=2, axis=0)


In [30]:
apt_dong_transaction_count_and_E = \
apt_dong_transaction_count_and_E[['NO', 'contract_date', 'USD_exchange_rate',
       'CNY_exchange_rate', 'JPY_exchange_rate', 'gu', 'a_dong', 'l_dong',
       'gu_adong', 'gu_ldong', 'adong_prev_1_month', 'adong_prev_3_months',
       'adong_prev_6_months', 'ldong_prev_1_month', 'ldong_prev_3_months',
       'ldong_prev_6_months']]

In [31]:
apt_dong_transaction_count_and_E.to_csv('Data_Preprocessing/apt_dong_transaction_count_and_E.csv', index=False, encoding='utf-8')

In [32]:
apt_info_with_distance = pd.read_csv('Data_Preprocessing/apt_information_with_distance_and_not_distance.csv')
politics_df = pd.read_csv('정치.csv')

In [33]:
politics_df.columns = ['date', 'K_progressive_president', 'K_conservative_president',
                       'K_progressive_majority', 'K_conservative_majority',
                       'A_progressive_president', 'A_conservative_president',
                       'A_progressive_senate', 'A_conservative_senate', 'A_progressive_house',
                       'A_conservative_house']
politics_df['date'] = pd.to_datetime(politics_df['date'])

In [34]:
apt_info_with_distance.info()
politics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   NO                        251096 non-null  object 
 1   contract_date             251096 non-null  object 
 2   closest_bus_stop_dist_km  251096 non-null  float64
 3   bus_stop_score            251096 non-null  int64  
 4   closest_park_dist_km      251096 non-null  float64
 5   park_score                251096 non-null  int64  
 6   closest_hospital_dist_km  251096 non-null  float64
 7   hospital_score            251096 non-null  int64  
 8   closest_school_dist_km    251096 non-null  float64
 9   school_score              251096 non-null  int64  
 10  closest_station_dist_km   251096 non-null  float64
 11  station_score             251096 non-null  int64  
 12  total_score               251096 non-null  int64  
 13  latitude                  251096 non-null  f

In [35]:
apt_info_with_distance['contract_date'] = pd.to_datetime(apt_info_with_distance['contract_date'])
apt_info_with_distance.info(), apt_info_with_distance.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   NO                        251096 non-null  object        
 1   contract_date             251096 non-null  datetime64[ns]
 2   closest_bus_stop_dist_km  251096 non-null  float64       
 3   bus_stop_score            251096 non-null  int64         
 4   closest_park_dist_km      251096 non-null  float64       
 5   park_score                251096 non-null  int64         
 6   closest_hospital_dist_km  251096 non-null  float64       
 7   hospital_score            251096 non-null  int64         
 8   closest_school_dist_km    251096 non-null  float64       
 9   school_score              251096 non-null  int64         
 10  closest_station_dist_km   251096 non-null  float64       
 11  station_score             251096 non-null  int64         
 12  to

(None,
            NO contract_date  closest_bus_stop_dist_km  bus_stop_score  \
 0  2019_00001    2019-12-31                  0.177637             509   
 1  2019_00002    2019-12-31                  0.139836             445   
 2  2019_00003    2019-12-31                  0.214917             375   
 3  2019_00004    2019-12-31                  0.147960             336   
 4  2019_00005    2019-12-31                  0.208754             493   
 
    closest_park_dist_km  park_score  closest_hospital_dist_km  hospital_score  \
 0              0.493332          15                  0.088037             682   
 1              1.288785           1                  0.431829             514   
 2              0.285478          16                  0.000000             977   
 3              0.880074           5                  0.125651             119   
 4              1.279134           2                  0.199009             785   
 
    closest_school_dist_km  school_score  ...   longi

In [36]:
merged_df = pd.merge(apt_info_with_distance, politics_df, how='outer', left_on='contract_date', right_on='date')

In [37]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251097 entries, 0 to 251096
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   NO                        251096 non-null  object        
 1   contract_date             251096 non-null  datetime64[ns]
 2   closest_bus_stop_dist_km  251096 non-null  float64       
 3   bus_stop_score            251096 non-null  float64       
 4   closest_park_dist_km      251096 non-null  float64       
 5   park_score                251096 non-null  float64       
 6   closest_hospital_dist_km  251096 non-null  float64       
 7   hospital_score            251096 non-null  float64       
 8   closest_school_dist_km    251096 non-null  float64       
 9   school_score              251096 non-null  float64       
 10  closest_station_dist_km   251096 non-null  float64       
 11  station_score             251096 non-null  float64       
 12  to

In [38]:
merged_df.isna().sum()

NO                          1
contract_date               1
closest_bus_stop_dist_km    1
bus_stop_score              1
closest_park_dist_km        1
park_score                  1
closest_hospital_dist_km    1
hospital_score              1
closest_school_dist_km      1
school_score                1
closest_station_dist_km     1
station_score               1
total_score                 1
latitude                    1
longitude                   1
exclusive_area_m2           1
floor                       1
brand_label_encoded         1
Q_1                         1
Q_2                         1
Q_3                         1
Q_4                         1
apt_age                     1
adjusted_price              1
date                        0
K_progressive_president     0
K_conservative_president    0
K_progressive_majority      0
K_conservative_majority     0
A_progressive_president     0
A_conservative_president    0
A_progressive_senate        0
A_conservative_senate       0
A_progress

In [39]:
pd.set_option('display.max_columns', None)  # 모든 컬럼 표시
pd.set_option('display.width', None)  # 화면 너비 제한 해제
print(merged_df[merged_df['NO'].isnull()])

         NO contract_date  closest_bus_stop_dist_km  bus_stop_score  \
212740  NaN           NaT                       NaN             NaN   

        closest_park_dist_km  park_score  closest_hospital_dist_km  \
212740                   NaN         NaN                       NaN   

        hospital_score  closest_school_dist_km  school_score  \
212740             NaN                     NaN           NaN   

        closest_station_dist_km  station_score  total_score  latitude  \
212740                      NaN            NaN          NaN       NaN   

        longitude  exclusive_area_m2  floor  brand_label_encoded  Q_1  Q_2  \
212740        NaN                NaN    NaN                  NaN  NaN  NaN   

        Q_3  Q_4  apt_age  adjusted_price       date  K_progressive_president  \
212740  NaN  NaN      NaN             NaN 2022-09-11                        0   

        K_conservative_president  K_progressive_majority  \
212740                         1                       1   


In [40]:
result = merged_df[(merged_df['contract_date'].dt.year == 2022) & 
                 (merged_df['contract_date'].dt.month == 9) & 
                 (merged_df['contract_date'].dt.day == 11)]
result

# merged_df = merged_df.dropna(subset='NO')
# merged_df.isna().sum()

,NO,contract_date,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,total_score,latitude,longitude,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,adjusted_price,date,K_progressive_president,K_conservative_president,K_progressive_majority,K_conservative_majority,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house


In [41]:
# 2019년 ~ 2023년 한국의 총선결과 다수당이 진보로 일관 되어있어 지움
merged_df = merged_df.drop(columns=['K_progressive_majority','K_conservative_majority'])

In [42]:
# 평당가격
# 'adjusted_price'를 'exclusive_area_m2'로 나누고 3.3058을 곱해 평으로 변환
merged_df['pyeong_price'] = merged_df['adjusted_price'] / (merged_df['exclusive_area_m2'] / 3.3058)

In [43]:
apt_dong_transaction_count_and_E.columns

Index(['NO', 'contract_date', 'USD_exchange_rate', 'CNY_exchange_rate',
       'JPY_exchange_rate', 'gu', 'a_dong', 'l_dong', 'gu_adong', 'gu_ldong',
       'adong_prev_1_month', 'adong_prev_3_months', 'adong_prev_6_months',
       'ldong_prev_1_month', 'ldong_prev_3_months', 'ldong_prev_6_months'],
      dtype='object')

In [44]:
merged_df.columns

Index(['NO', 'contract_date', 'closest_bus_stop_dist_km', 'bus_stop_score',
       'closest_park_dist_km', 'park_score', 'closest_hospital_dist_km',
       'hospital_score', 'closest_school_dist_km', 'school_score',
       'closest_station_dist_km', 'station_score', 'total_score', 'latitude',
       'longitude', 'exclusive_area_m2', 'floor', 'brand_label_encoded', 'Q_1',
       'Q_2', 'Q_3', 'Q_4', 'apt_age', 'adjusted_price', 'date',
       'K_progressive_president', 'K_conservative_president',
       'A_progressive_president', 'A_conservative_president',
       'A_progressive_senate', 'A_conservative_senate', 'A_progressive_house',
       'A_conservative_house', 'pyeong_price'],
      dtype='object')

In [45]:
merged_aptinfo_distance_politics_transactionCount_E = \
pd.merge(apt_dong_transaction_count_and_E[['NO', 'USD_exchange_rate', 'CNY_exchange_rate',
       'JPY_exchange_rate', 'gu', 'a_dong', 'l_dong', 'gu_adong', 'gu_ldong',
       'adong_prev_1_month', 'adong_prev_3_months', 'adong_prev_6_months',
       'ldong_prev_1_month', 'ldong_prev_3_months', 'ldong_prev_6_months']], 
         merged_df, how='inner', on='NO')

# 'NO' 컬럼을 기준으로 오름차순으로 정렬
merged_aptinfo_distance_politics_transactionCount_E_sorted = merged_aptinfo_distance_politics_transactionCount_E.sort_values(by='NO', ascending=True)

# 결과 확인
print(merged_aptinfo_distance_politics_transactionCount_E_sorted.head())
print(merged_aptinfo_distance_politics_transactionCount_E_sorted.tail())
merged_aptinfo_distance_politics_transactionCount_E_sorted.info()

               NO  USD_exchange_rate  CNY_exchange_rate  JPY_exchange_rate  \
51975  2019_00001             1157.8             165.74            1063.47   
51976  2019_00002             1157.8             165.74            1063.47   
51977  2019_00003             1157.8             165.74            1063.47   
51978  2019_00004             1157.8             165.74            1063.47   
51979  2019_00005             1157.8             165.74            1063.47   

         gu a_dong l_dong    gu_adong   gu_ldong  adong_prev_1_month  \
51975   성동구    마장동    마장동     성동구_마장동    성동구_마장동                  45   
51976   중랑구   면목2동    면목동    중랑구_면목2동    중랑구_면목동                  10   
51977  동대문구  답십리2동   답십리동  동대문구_답십리2동  동대문구_답십리동                  46   
51978   성북구   정릉4동    정릉동    성북구_정릉4동    성북구_정릉동                  35   
51979  동대문구   장안2동    장안동   동대문구_장안2동   동대문구_장안동                   2   

       adong_prev_3_months  adong_prev_6_months  ldong_prev_1_month  \
51975                   96 

In [46]:
merged_aptinfo_distance_politics_transactionCount_E_sorted.isna().sum()

NO                          0
USD_exchange_rate           0
CNY_exchange_rate           0
JPY_exchange_rate           0
gu                          0
a_dong                      0
l_dong                      0
gu_adong                    0
gu_ldong                    0
adong_prev_1_month          0
adong_prev_3_months         0
adong_prev_6_months         0
ldong_prev_1_month          0
ldong_prev_3_months         0
ldong_prev_6_months         0
contract_date               0
closest_bus_stop_dist_km    0
bus_stop_score              0
closest_park_dist_km        0
park_score                  0
closest_hospital_dist_km    0
hospital_score              0
closest_school_dist_km      0
school_score                0
closest_station_dist_km     0
station_score               0
total_score                 0
latitude                    0
longitude                   0
exclusive_area_m2           0
floor                       0
brand_label_encoded         0
Q_1                         0
Q_2       

In [47]:
merged_aptinfo_distance_politics_transactionCount_E_sorted.to_csv('Data_Preprocessing/merged_aptinfo_distance_politics_transactionCount_E_sorted.csv', index=False, encoding='utf-8')

In [48]:
merged_aptinfo_distance_politics_transactionCount_E_sorted[merged_aptinfo_distance_politics_transactionCount_E_sorted['a_dong'].str.contains('종로')]

,NO,USD_exchange_rate,CNY_exchange_rate,JPY_exchange_rate,gu,a_dong,l_dong,gu_adong,gu_ldong,adong_prev_1_month,adong_prev_3_months,adong_prev_6_months,ldong_prev_1_month,ldong_prev_3_months,ldong_prev_6_months,contract_date,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,total_score,latitude,longitude,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,adjusted_price,date,K_progressive_president,K_conservative_president,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house,pyeong_price
51558,2019_00457,1160.9,165.87,1060.72,종로구,종로1·2·3·4가동,인의동,종로구_종로1·2·3·4가동,종로구_인의동,1,4,9,1,5,11,2019-12-30,0.080981,567.0,0.751104,8.0,0.051713,1245.0,0.594715,21.0,0.396668,58.0,1899.0,37.571448,126.998320,98.39,16.0,1.0,0.0,0.0,0.0,1.0,13.0,94439.396985,2019-12-30,1,0,0,1,0,1,1,0,3173.063915
50586,2019_02104,1162.6,166.14,1061.98,종로구,종로1·2·3·4가동,인의동,종로구_종로1·2·3·4가동,종로구_인의동,1,4,9,1,5,11,2019-12-23,0.080981,567.0,0.751104,8.0,0.051713,1245.0,0.594715,21.0,0.396668,58.0,1899.0,37.571448,126.998320,84.98,14.0,1.0,0.0,0.0,0.0,1.0,13.0,76830.150754,2019-12-23,1,0,0,1,0,1,1,0,2988.763384
194681,2019_04231,1289.4,180.84,912.66,종로구,종로1·2·3·4가동,인의동,종로구_종로1·2·3·4가동,종로구_인의동,1,4,9,1,5,11,2019-12-14,0.080981,567.0,0.751104,8.0,0.051713,1245.0,0.594715,21.0,0.396668,58.0,1899.0,37.571448,126.998320,84.64,11.0,1.0,0.0,0.0,0.0,1.0,13.0,80755.778894,2019-12-14,1,0,0,1,0,1,1,0,3154.093264
48437,2019_05680,1193.3,169.64,1099.36,종로구,종로1·2·3·4가동,익선동,종로구_종로1·2·3·4가동,종로구_익선동,1,4,9,2,6,8,2019-12-12,0.144267,573.0,0.497353,11.0,0.102414,986.0,0.220735,52.0,0.287302,50.0,1672.0,37.575112,126.990058,48.54,2.0,1.0,0.0,0.0,0.0,1.0,16.0,34209.045226,2019-12-12,1,0,0,1,0,1,1,0,2329.795256
47389,2019_06542,1189.7,168.88,1095.49,종로구,종로1·2·3·4가동,인의동,종로구_종로1·2·3·4가동,종로구_인의동,1,4,9,1,5,11,2019-12-09,0.080981,567.0,0.751104,8.0,0.051713,1245.0,0.594715,21.0,0.396668,58.0,1899.0,37.571448,126.998320,106.81,13.0,1.0,0.0,0.0,0.0,1.0,13.0,95336.683417,2019-12-09,1,0,0,1,0,1,1,0,2950.697576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242412,2023_27533,1289.4,180.84,912.66,종로구,종로1·2·3·4가동,인의동,종로구_종로1·2·3·4가동,종로구_인의동,1,3,10,1,3,7,2023-04-09,0.080981,567.0,0.751104,8.0,0.051713,1245.0,0.594715,21.0,0.396668,55.0,1896.0,37.571448,126.998320,92.81,14.0,1.0,0.0,1.0,0.0,0.0,17.0,90000.000000,2023-04-09,0,1,1,0,1,0,0,1,3205.710592
153475,2023_29429,1307.8,190.05,988.62,종로구,종로1·2·3·4가동,익선동,종로구_종로1·2·3·4가동,종로구_익선동,1,5,10,1,7,12,2023-03-22,0.144267,573.0,0.497353,11.0,0.102414,986.0,0.220735,52.0,0.287302,50.0,1672.0,37.575112,126.990058,46.64,5.0,1.0,1.0,0.0,0.0,0.0,20.0,38200.000000,2023-03-22,0,1,1,0,1,0,0,1,2707.580617
151926,2023_31675,1317.4,188.79,967.36,종로구,종로5·6가동,연지동,종로구_종로5·6가동,종로구_연지동,2,4,8,1,2,2,2023-02-28,0.153803,570.0,0.845647,12.0,0.098834,1198.0,0.534740,22.0,0.427713,46.0,1848.0,37.573174,126.998728,47.24,13.0,1.0,1.0,0.0,0.0,0.0,8.0,38000.000000,2023-02-28,0,1,1,0,1,0,0,1,2659.195597
150539,2023_33638,1247.5,183.46,940.34,종로구,종로5·6가동,충신동,종로구_종로5·6가동,종로구_충신동,1,4,7,1,3,6,2023-02-07,0.123863,540.0,0.582935,15.0,0.161535,1215.0,0.274759,37.0,0.528724,27.0,1834.0,37.575141,127.004050,21.94,15.0,1.0,1.0,0.0,0.0,0.0,12.0,17000.000000,2023-02-07,0,1,1,0,1,0,0,1,2561.467639


In [49]:
pop_density_df = pd.read_csv('forDP/mean_people.csv', encoding='cp949')

In [50]:
pop_density_df

,동별(1),동별(2),동별(3),2019,2020,2021,2022,2023
0,동별(1),동별(2),동별(3),인구밀도 (명/㎢),인구밀도 (명/㎢),인구밀도 (명/㎢),인구밀도 (명/㎢),인구밀도 (명/㎢)
1,합계,소계,소계,16541,16376,16086,15974,15927
2,합계,종로구,소계,6769,6649,6431,6365,6292
3,합계,종로구,사직동,7980,7972,7834,7606,7539
4,합계,종로구,삼청동,2011,1952,1838,1773,1649
...,...,...,...,...,...,...,...,...
451,합계,강동구,천호2동,22869,22453,21499,21545,21290
452,합계,강동구,길동,21367,21598,28555,28183,27976
453,합계,강동구,강일동,14797,14797,14594,14627,15169
454,합계,강동구,상일1동,21529,21529,21390,21526,21670


In [51]:
pop_density_df.columns = \
    ['합계', 'gu', 'a_dong', 'population_density_2019',
      'population_density_2020',
      'population_density_2021',
      'population_density_2022',
      'population_density_2023']
pop_density_df['gu_adong'] = pop_density_df['gu'] + '_' + pop_density_df['a_dong']

In [52]:
pop_density_df.isna().sum()

합계                         0
gu                         0
a_dong                     0
population_density_2019    0
population_density_2020    0
population_density_2021    0
population_density_2022    0
population_density_2023    0
gu_adong                   0
dtype: int64

In [53]:
# '소계'를 포함하는 행을 제외하기
pop_density_df = pop_density_df[~pop_density_df['gu_adong'].str.contains('소계', na=False)]
pop_density_df = pop_density_df.drop(index=0)  # 0번째 행 제거

# 결과 확인
print(pop_density_df.head())


   합계   gu a_dong population_density_2019 population_density_2020  \
3  합계  종로구    사직동                    7980                    7972   
4  합계  종로구    삼청동                    2011                    1952   
5  합계  종로구    부암동                    4648                    4552   
6  합계  종로구    평창동                    2121                    2095   
7  합계  종로구    무악동                   24278                   23867   

  population_density_2021 population_density_2022 population_density_2023  \
3                    7834                    7606                    7539   
4                    1838                    1773                    1649   
5                    4309                    4201                    4136   
6                    2066                    2013                    1982   
7                   23047                   22367                   22197   

  gu_adong  
3  종로구_사직동  
4  종로구_삼청동  
5  종로구_부암동  
6  종로구_평창동  
7  종로구_무악동  


In [54]:
pop_density_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 3 to 455
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   합계                       429 non-null    object
 1   gu                       429 non-null    object
 2   a_dong                   429 non-null    object
 3   population_density_2019  429 non-null    object
 4   population_density_2020  429 non-null    object
 5   population_density_2021  429 non-null    object
 6   population_density_2022  429 non-null    object
 7   population_density_2023  429 non-null    object
 8   gu_adong                 429 non-null    object
dtypes: object(9)
memory usage: 33.5+ KB


In [55]:
pop_density_df = pop_density_df[['gu_adong', 'gu', 'a_dong', 
       'population_density_2019', 
       'population_density_2020', 
       'population_density_2021', 
       'population_density_2022', 
       'population_density_2023']]

In [56]:
pop_density_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 3 to 455
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   gu_adong                 429 non-null    object
 1   gu                       429 non-null    object
 2   a_dong                   429 non-null    object
 3   population_density_2019  429 non-null    object
 4   population_density_2020  429 non-null    object
 5   population_density_2021  429 non-null    object
 6   population_density_2022  429 non-null    object
 7   population_density_2023  429 non-null    object
dtypes: object(8)
memory usage: 30.2+ KB


In [57]:
# 방법 1: 개별 컬럼 변환
columns_to_convert = [
    'population_density_2019',
     'population_density_2020',
     'population_density_2021',
     'population_density_2022']

# '-' 값을 0으로 대체
pop_density_df[columns_to_convert] = pop_density_df[columns_to_convert].replace('-', '0')

# 소수형으로 변환
for col in columns_to_convert:
    pop_density_df[col] = pop_density_df[col].str.replace(',', '').astype(float)
    
pop_density_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 3 to 455
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gu_adong                 429 non-null    object 
 1   gu                       429 non-null    object 
 2   a_dong                   429 non-null    object 
 3   population_density_2019  429 non-null    float64
 4   population_density_2020  429 non-null    float64
 5   population_density_2021  429 non-null    float64
 6   population_density_2022  429 non-null    float64
 7   population_density_2023  429 non-null    object 
dtypes: float64(4), object(4)
memory usage: 30.2+ KB


In [58]:
pop_density_df['population_density_2023'] = pop_density_df['population_density_2023'].replace('-', '0')
pop_density_df['population_density_2023'] = pop_density_df['population_density_2023'].astype('float')

In [59]:
pop_density_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 3 to 455
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gu_adong                 429 non-null    object 
 1   gu                       429 non-null    object 
 2   a_dong                   429 non-null    object 
 3   population_density_2019  429 non-null    float64
 4   population_density_2020  429 non-null    float64
 5   population_density_2021  429 non-null    float64
 6   population_density_2022  429 non-null    float64
 7   population_density_2023  429 non-null    float64
dtypes: float64(5), object(3)
memory usage: 30.2+ KB


In [60]:
gu_and_adong = pd.read_csv('../4.Public_transportation/Gu_and_Administrative_dong.csv', encoding='cp949')
gu_and_ldong = pd.read_csv('../4.Public_transportation/Gu_and_Legal_dong.csv', encoding='cp949')

In [61]:
gu_and_adong.columns = ['gu', 'a_dong']
gu_and_adong['gu_dong'] = gu_and_adong['gu'] + '_' + gu_and_adong['a_dong']
gu_and_adong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   gu       426 non-null    object
 1   a_dong   426 non-null    object
 2   gu_dong  426 non-null    object
dtypes: object(3)
memory usage: 10.1+ KB


In [62]:
gu_and_adong['gu_dong'] = gu_and_adong['gu_dong'].str.replace(' ', '')

In [63]:
gu_and_adong['gu_dong'] = gu_and_adong['gu_dong'].replace('관악구_보라매', '관악구_보라매동')

In [64]:
# 두 컬럼의 값을 집합으로 변환
gu_adong_set = set(pop_density_df["gu_adong"])
gu_dong_set = set(gu_and_adong["gu_dong"])

# gu_adong에만 있고 gu_dong에 없는 값
only_in_gu_adong = gu_adong_set - gu_dong_set

# gu_dong에만 있고 gu_adong에 없는 값
only_in_gu_dong = gu_dong_set - gu_adong_set

# 결과 출력
print("gu_adong에만 있는 값:", only_in_gu_adong)
print("gu_dong에만 있는 값:", only_in_gu_dong)


gu_adong에만 있는 값: {'강동구_상일동', '강남구_일원2동'}
gu_dong에만 있는 값: set()


In [65]:
pop_density_df[pop_density_df['gu_adong'].str.contains('상일동')]
pop_density_df[pop_density_df['gu_adong'].str.contains('일원2동')]

,gu_adong,gu,a_dong,population_density_2019,population_density_2020,population_density_2021,population_density_2022,population_density_2023
399,강남구_일원2동,강남구,일원2동,14645.0,14281.0,14181.0,14369.0,14369.0


In [66]:
# 강동구 상일동을 상일1동으로 변경
pop_density_df['gu_adong'] = pop_density_df['gu_adong'].replace('강동구_상일동', '강동구_상일1동')

# 강남구 일원2동을 일원본동으로 변경
pop_density_df['gu_adong'] = pop_density_df['gu_adong'].replace('강남구_일원2동', '강남구_일원본동')

In [67]:
# 두 컬럼의 값을 집합으로 변환
gu_adong_set = set(pop_density_df["gu_adong"])
gu_dong_set = set(gu_and_adong["gu_dong"])

# gu_adong에만 있고 gu_dong에 없는 값
only_in_gu_adong = gu_adong_set - gu_dong_set

# gu_dong에만 있고 gu_adong에 없는 값
only_in_gu_dong = gu_dong_set - gu_adong_set

# 결과 출력
print("gu_adong에만 있는 값:", only_in_gu_adong)
print("gu_dong에만 있는 값:", only_in_gu_dong)


gu_adong에만 있는 값: set()
gu_dong에만 있는 값: set()


In [68]:
pop_density_df.columns
pop_density_2019 = pop_density_df[['gu_adong', 'gu', 'a_dong', 'population_density_2019']]
pop_density_2020 = pop_density_df[['gu_adong', 'gu', 'a_dong',  'population_density_2020']]
pop_density_2021 = pop_density_df[['gu_adong', 'gu', 'a_dong',  'population_density_2021']]
pop_density_2022 = pop_density_df[['gu_adong', 'gu', 'a_dong',  'population_density_2022']]
pop_density_2023 = pop_density_df[['gu_adong', 'gu', 'a_dong',  'population_density_2023']]

In [69]:
pop_density_2019.info(), pop_density_2020.info(), pop_density_2021.info(), \
    pop_density_2022.info(), pop_density_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 3 to 455
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gu_adong                 429 non-null    object 
 1   gu                       429 non-null    object 
 2   a_dong                   429 non-null    object 
 3   population_density_2019  429 non-null    float64
dtypes: float64(1), object(3)
memory usage: 16.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 3 to 455
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gu_adong                 429 non-null    object 
 1   gu                       429 non-null    object 
 2   a_dong                   429 non-null    object 
 3   population_density_2020  429 non-null    float64
dtypes: float64(1), object(3)
memory usage: 16.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 429

(None, None, None, None, None)

In [70]:
# 모든 컬럼 기준으로 중복 확인 1
# 2019
duplicates_19 = pop_density_2019[pop_density_2019.duplicated(subset=['gu_adong'], keep=False)]
print('19년 중복:\n', duplicates_19)

# 2020
duplicates_20 = pop_density_2020[pop_density_2020.duplicated(subset=['gu_adong'], keep=False)]
print('\n20년 중복:\n', duplicates_20)

# 2021
duplicates_21 = pop_density_2021[pop_density_2021.duplicated(subset=['gu_adong'], keep=False)]
print('\n21년 중복:\n', duplicates_21)

# 2022
duplicates_22 = pop_density_2022[pop_density_2022.duplicated(subset=['gu_adong'], keep=False)]
print('\n22년 중복:\n', duplicates_22)

# 2023
duplicates_23 = pop_density_2023[pop_density_2023.duplicated(subset=['gu_adong'], keep=False)]
print('\n23년 중복:\n', duplicates_23)


19년 중복:
      gu_adong   gu a_dong  population_density_2019
397  강남구_일원본동  강남구   일원본동                   9457.0
399  강남구_일원본동  강남구   일원2동                  14645.0
435   강동구_강일동  강동구    강일동                  11474.0
436  강동구_상일1동  강동구    상일동                   4065.0
453   강동구_강일동  강동구    강일동                  14797.0
454  강동구_상일1동  강동구   상일1동                  21529.0

20년 중복:
      gu_adong   gu a_dong  population_density_2020
397  강남구_일원본동  강남구   일원본동                   9281.0
399  강남구_일원본동  강남구   일원2동                  14281.0
435   강동구_강일동  강동구    강일동                  12778.0
436  강동구_상일1동  강동구    상일동                  12583.0
453   강동구_강일동  강동구    강일동                  14797.0
454  강동구_상일1동  강동구   상일1동                  21529.0

21년 중복:
      gu_adong   gu a_dong  population_density_2021
397  강남구_일원본동  강남구   일원본동                   9028.0
399  강남구_일원본동  강남구   일원2동                  14181.0
435   강동구_강일동  강동구    강일동                  12126.0
436  강동구_상일1동  강동구    상일동                   8324.0
45

In [71]:
# 0 값을 NaN으로 바꾸고 해당 행 제거
pop_density_2019 = pop_density_2019.replace({'population_density_2019': 0}, np.nan).dropna(subset=['population_density_2019'])
pop_density_2020 = pop_density_2020.replace({'population_density_2020': 0}, np.nan).dropna(subset=['population_density_2020'])
pop_density_2021 = pop_density_2021.replace({'population_density_2021': 0}, np.nan).dropna(subset=['population_density_2021'])
pop_density_2022 = pop_density_2022.replace({'population_density_2022': 0}, np.nan).dropna(subset=['population_density_2022'])
pop_density_2023 = pop_density_2023.replace({'population_density_2023': 0}, np.nan).dropna(subset=['population_density_2023'])


In [72]:
# 모든 컬럼 기준으로 중복 확인 2
# 2019
duplicates_19 = pop_density_2019[pop_density_2019.duplicated(subset=['gu_adong'], keep=False)]
print('19년 중복:\n', duplicates_19)

# 2020
duplicates_20 = pop_density_2020[pop_density_2020.duplicated(subset=['gu_adong'], keep=False)]
print('\n20년 중복:\n', duplicates_20)

# 2021
duplicates_21 = pop_density_2021[pop_density_2021.duplicated(subset=['gu_adong'], keep=False)]
print('\n21년 중복:\n', duplicates_21)

# 2022
duplicates_22 = pop_density_2022[pop_density_2022.duplicated(subset=['gu_adong'], keep=False)]
print('\n22년 중복:\n', duplicates_22)

# 2023
duplicates_23 = pop_density_2023[pop_density_2023.duplicated(subset=['gu_adong'], keep=False)]
print('\n23년 중복:\n', duplicates_23)


19년 중복:
      gu_adong   gu a_dong  population_density_2019
397  강남구_일원본동  강남구   일원본동                   9457.0
399  강남구_일원본동  강남구   일원2동                  14645.0
435   강동구_강일동  강동구    강일동                  11474.0
436  강동구_상일1동  강동구    상일동                   4065.0
453   강동구_강일동  강동구    강일동                  14797.0
454  강동구_상일1동  강동구   상일1동                  21529.0

20년 중복:
      gu_adong   gu a_dong  population_density_2020
397  강남구_일원본동  강남구   일원본동                   9281.0
399  강남구_일원본동  강남구   일원2동                  14281.0
435   강동구_강일동  강동구    강일동                  12778.0
436  강동구_상일1동  강동구    상일동                  12583.0
453   강동구_강일동  강동구    강일동                  14797.0
454  강동구_상일1동  강동구   상일1동                  21529.0

21년 중복:
      gu_adong   gu a_dong  population_density_2021
397  강남구_일원본동  강남구   일원본동                   9028.0
399  강남구_일원본동  강남구   일원2동                  14181.0
435   강동구_강일동  강동구    강일동                  12126.0
436  강동구_상일1동  강동구    상일동                   8324.0
45

In [73]:
# def merge_rows(df, base_dong, merge_dong, year):
#     # 마스크 생성
#     mask_base = df['a_dong'].str.contains(base_dong)
#     mask_merge = df['a_dong'].str.contains(merge_dong)

#     # 해당 행 추출
#     rows_base = df[mask_base]
#     rows_merge = df[mask_merge]

#     # 인구, 면적, 인구밀도 합산 및 업데이트
#     if not rows_base.empty and not rows_merge.empty:
#         total_population = rows_base[f'population_{year}'].values[0] + rows_merge[f'population_{year}'].values[0]
#         total_area = rows_base[f'area_{year}'].values[0] + rows_merge[f'area_{year}'].values[0]
#         total_density = total_population / total_area

#         # base_dong 행 업데이트
#         df.loc[mask_base, f'population_{year}'] = total_population
#         df.loc[mask_base, f'area_{year}'] = total_area
#         df.loc[mask_base, f'population_density_{year}'] = total_density

#         # merge_dong 행 제거
#         df = df[~mask_merge].reset_index(drop=True)

#     return df

# # 데이터프레임 각각에 대해 실행
# for year, df_name in zip([2019, 2020, 2021], ['pop_density_2019', 'pop_density_2020', 'pop_density_2021']):
#     globals()[df_name] = merge_rows(globals()[df_name], '일원본동', '일원2동', year)


In [74]:
# 2019~2023 범위를 가진 year 컬럼 추가 및 데이터 타입을 int8로 설정
for year in range(2019, 2024):
    df_name = f'pop_density_{year}'
    globals()[df_name]['year'] = year  # year 컬럼 추가
    globals()[df_name]['year'] = globals()[df_name]['year'].astype('int16')  # 데이터 타입을 int8로 변환


In [75]:
pop_density_2019.info(), \
pop_density_2020.info(), \
pop_density_2021.info(), \
pop_density_2022.info(), \
pop_density_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 3 to 455
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gu_adong                 429 non-null    object 
 1   gu                       429 non-null    object 
 2   a_dong                   429 non-null    object 
 3   population_density_2019  429 non-null    float64
 4   year                     429 non-null    int16  
dtypes: float64(1), int16(1), object(3)
memory usage: 17.6+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 3 to 455
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   gu_adong                 429 non-null    object 
 1   gu                       429 non-null    object 
 2   a_dong                   429 non-null    object 
 3   population_density_2020  429 non-null    float64
 4   year                     429 non

(None, None, None, None, None)

In [76]:
merged_aptinfo_distance_politics_transactionCount_E_sorted.columns

Index(['NO', 'USD_exchange_rate', 'CNY_exchange_rate', 'JPY_exchange_rate',
       'gu', 'a_dong', 'l_dong', 'gu_adong', 'gu_ldong', 'adong_prev_1_month',
       'adong_prev_3_months', 'adong_prev_6_months', 'ldong_prev_1_month',
       'ldong_prev_3_months', 'ldong_prev_6_months', 'contract_date',
       'closest_bus_stop_dist_km', 'bus_stop_score', 'closest_park_dist_km',
       'park_score', 'closest_hospital_dist_km', 'hospital_score',
       'closest_school_dist_km', 'school_score', 'closest_station_dist_km',
       'station_score', 'total_score', 'latitude', 'longitude',
       'exclusive_area_m2', 'floor', 'brand_label_encoded', 'Q_1', 'Q_2',
       'Q_3', 'Q_4', 'apt_age', 'adjusted_price', 'date',
       'K_progressive_president', 'K_conservative_president',
       'A_progressive_president', 'A_conservative_president',
       'A_progressive_senate', 'A_conservative_senate', 'A_progressive_house',
       'A_conservative_house', 'pyeong_price'],
      dtype='object')

In [77]:
merged_aptinfo_distance_politics_transactionCount_E_sorted = \
merged_aptinfo_distance_politics_transactionCount_E_sorted[[
        'NO', 'contract_date', 'gu', 'a_dong', 'l_dong', 'gu_adong', 'gu_ldong', 'latitude', 
        'longitude', 'adong_prev_1_month', 'adong_prev_3_months', 'adong_prev_6_months', 
        'ldong_prev_1_month', 'ldong_prev_3_months', 'ldong_prev_6_months', 
        'closest_bus_stop_dist_km', 'bus_stop_score', 'closest_park_dist_km', 'park_score', 
        'closest_hospital_dist_km', 'hospital_score', 'closest_school_dist_km', 'school_score', 
        'closest_station_dist_km', 'station_score',  'exclusive_area_m2', 'floor', 
        'brand_label_encoded', 'Q_1', 'Q_2', 'Q_3', 'Q_4', 'apt_age', 
        'USD_exchange_rate', 'CNY_exchange_rate', 'JPY_exchange_rate',
        'K_progressive_president', 'K_conservative_president',
        'A_progressive_president', 'A_conservative_president',
        'A_progressive_senate', 'A_conservative_senate', 'A_progressive_house',
        'A_conservative_house', 'pyeong_price', 'adjusted_price'
]]

# 제외 'total_score', 'date'

In [78]:
merged_aptinfo_distance_politics_transactionCount_E_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 251096 entries, 51975 to 239882
Data columns (total 46 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   NO                        251096 non-null  object        
 1   contract_date             251096 non-null  datetime64[ns]
 2   gu                        251096 non-null  object        
 3   a_dong                    251096 non-null  object        
 4   l_dong                    251096 non-null  object        
 5   gu_adong                  251096 non-null  object        
 6   gu_ldong                  251096 non-null  object        
 7   latitude                  251096 non-null  float64       
 8   longitude                 251096 non-null  float64       
 9   adong_prev_1_month        251096 non-null  int16         
 10  adong_prev_3_months       251096 non-null  int16         
 11  adong_prev_6_months       251096 non-null  int16         
 12  ldo

In [79]:
merged_aptinfo_distance_politics_transactionCount_E_sorted.to_csv('Data_Preprocessing/merged_aptinfo_distance_politics_transactionCount_E_sorted.csv', index=False, encoding='utf-8')

In [80]:
merged_aptinfo_distance_politics_transactionCount_E_df = pd.read_csv('Data_Preprocessing/merged_aptinfo_distance_politics_transactionCount_E_sorted.csv')

In [81]:
merged_aptinfo_distance_politics_transactionCount_E_df

,NO,contract_date,gu,a_dong,l_dong,gu_adong,gu_ldong,latitude,longitude,adong_prev_1_month,adong_prev_3_months,adong_prev_6_months,ldong_prev_1_month,ldong_prev_3_months,ldong_prev_6_months,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,USD_exchange_rate,CNY_exchange_rate,JPY_exchange_rate,K_progressive_president,K_conservative_president,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house,pyeong_price,adjusted_price
0,2019_00001,2019-12-31,성동구,마장동,마장동,성동구_마장동,성동구_마장동,37.569628,127.041524,45,96,151,45,96,151,0.177637,509.0,0.493332,15.0,0.088037,682.0,0.143573,45.0,0.424042,37.0,134.790,17.0,1.0,0.0,0.0,0.0,1.0,21.0,1157.8,165.74,1063.47,1,0,0,1,0,1,1,0,2420.709575,98701.507538
1,2019_00002,2019-12-31,중랑구,면목2동,면목동,중랑구_면목2동,중랑구_면목동,37.589283,127.072500,10,32,63,3,26,48,0.139836,445.0,1.288785,1.0,0.431829,514.0,0.336639,65.0,0.744370,6.0,84.980,9.0,1.0,0.0,0.0,0.0,1.0,21.0,1157.8,165.74,1063.47,1,0,0,1,0,1,1,0,2085.589631,53612.864322
2,2019_00003,2019-12-31,동대문구,답십리2동,답십리동,동대문구_답십리2동,동대문구_답십리동,37.574609,127.060129,46,97,152,37,127,214,0.214917,375.0,0.285478,16.0,0.000000,977.0,0.233091,71.0,1.202505,2.0,59.740,21.0,1.0,0.0,0.0,0.0,1.0,19.0,1157.8,165.74,1063.47,1,0,0,1,0,1,1,0,3382.586982,61127.638191
3,2019_00004,2019-12-31,성북구,정릉4동,정릉동,성북구_정릉4동,성북구_정릉동,37.616686,127.001842,35,103,178,2,65,125,0.147960,336.0,0.880074,5.0,0.125651,119.0,0.536320,30.0,0.878111,6.0,71.500,2.0,1.0,0.0,0.0,0.0,1.0,42.0,1157.8,165.74,1063.47,1,0,0,1,0,1,1,0,1669.811774,36115.778894
4,2019_00005,2019-12-31,동대문구,장안2동,장안동,동대문구_장안2동,동대문구_장안동,37.576338,127.074573,2,66,123,49,163,254,0.208754,493.0,1.279134,2.0,0.199009,785.0,0.200016,42.0,1.384230,1.0,59.928,12.0,4.0,0.0,0.0,0.0,1.0,12.0,1157.8,165.74,1063.47,1,0,0,1,0,1,1,0,4052.557682,73465.326633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251091,2023_35638,2023-01-02,은평구,대조동,대조동,은평구_대조동,은평구_대조동,37.617576,126.921087,1,5,18,1,5,18,0.120570,681.0,0.989016,6.0,0.050423,1813.0,0.400552,53.0,0.128523,29.0,14.030,11.0,1.0,1.0,0.0,0.0,0.0,3.0,1267.3,181.44,968.55,0,1,1,0,1,0,1,0,4005.602281,17000.000000
251092,2023_35639,2023-01-02,은평구,대조동,대조동,은평구_대조동,은평구_대조동,37.609112,126.926435,1,5,18,1,5,18,0.047301,549.0,1.144110,1.0,0.056842,1480.0,0.367395,40.0,0.382468,30.0,29.990,9.0,1.0,1.0,0.0,0.0,0.0,2.0,1267.3,181.44,968.55,0,1,1,0,1,0,1,0,4409.203068,40000.000000
251093,2023_35640,2023-01-01,강남구,일원본동,일원동,강남구_일원본동,강남구_일원동,37.485531,127.084798,5,10,16,1,5,11,0.179996,295.0,0.780690,5.0,0.000000,356.0,0.179475,69.0,0.210601,12.0,99.790,6.0,1.0,1.0,0.0,0.0,0.0,30.0,1289.4,180.84,912.66,0,1,1,0,1,0,1,0,5797.324381,175000.000000
251094,2023_35641,2023-01-01,강남구,일원본동,일원동,강남구_일원본동,강남구_일원동,37.485531,127.084798,5,10,16,1,5,11,0.179996,295.0,0.780690,5.0,0.000000,356.0,0.179475,69.0,0.210601,12.0,99.790,6.0,1.0,1.0,0.0,0.0,0.0,30.0,1289.4,180.84,912.66,0,1,1,0,1,0,1,0,5797.324381,175000.000000


In [82]:
final = merged_aptinfo_distance_politics_transactionCount_E_df
final[final['gu_adong']=='관악구_신대방2동']


,NO,contract_date,gu,a_dong,l_dong,gu_adong,gu_ldong,latitude,longitude,adong_prev_1_month,adong_prev_3_months,adong_prev_6_months,ldong_prev_1_month,ldong_prev_3_months,ldong_prev_6_months,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,USD_exchange_rate,CNY_exchange_rate,JPY_exchange_rate,K_progressive_president,K_conservative_president,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house,pyeong_price,adjusted_price
1387,2019_01388,2019-12-26,관악구,신대방2동,봉천동,관악구_신대방2동,관악구_봉천동,37.491211,126.924990,1,6,10,35,47,166,0.038911,622.0,0.755521,5.0,0.000000,1127.0,0.342739,47.0,0.940023,8.0,59.94,5.0,1.0,0.0,0.0,0.0,1.0,23.0,1163.7,166.01,1063.76,1,0,0,1,0,1,1,0,2746.527303,49799.396985
9073,2019_09074,2019-12-02,관악구,신대방2동,봉천동,관악구_신대방2동,관악구_봉천동,37.491901,126.925413,1,6,10,35,47,166,0.071805,649.0,0.793633,6.0,0.049781,1165.0,0.259374,56.0,0.923866,12.0,142.47,21.0,2.0,0.0,0.0,0.0,1.0,19.0,1180.3,167.84,1077.75,1,0,0,1,0,1,1,0,2212.142964,95336.683417
11022,2019_11023,2019-11-28,관악구,신대방2동,봉천동,관악구_신대방2동,관악구_봉천동,37.491901,126.925413,2,6,10,65,78,162,0.071805,649.0,0.793633,6.0,0.049781,1165.0,0.259374,56.0,0.923866,12.0,183.40,19.0,2.0,0.0,0.0,0.0,1.0,19.0,1176.0,167.46,1075.30,1,0,0,1,0,1,1,0,1809.428361,100383.919598
25904,2019_25905,2019-10-18,관악구,신대방2동,봉천동,관악구_신대방2동,관악구_봉천동,37.491211,126.924990,3,6,9,83,91,125,0.038911,622.0,0.755521,5.0,0.000000,1127.0,0.342739,47.0,0.940023,8.0,59.94,16.0,1.0,0.0,0.0,0.0,1.0,23.0,1186.6,167.05,1092.99,1,0,0,1,0,1,1,0,2963.032834,53725.025126
26723,2019_26724,2019-10-16,관악구,신대방2동,봉천동,관악구_신대방2동,관악구_봉천동,37.491211,126.924990,3,6,9,83,91,125,0.038911,622.0,0.755521,5.0,0.000000,1127.0,0.342739,47.0,0.940023,8.0,59.94,17.0,1.0,0.0,0.0,0.0,1.0,23.0,1184.1,167.38,1087.93,1,0,0,1,0,1,1,0,3062.006791,55519.597990
35196,2019_35197,2019-09-23,관악구,신대방2동,봉천동,관악구_신대방2동,관악구_봉천동,37.491211,126.924990,1,4,7,32,39,99,0.038911,622.0,0.755521,5.0,0.000000,1127.0,0.342739,47.0,0.940023,8.0,59.94,12.0,1.0,0.0,0.0,1.0,0.0,23.0,1191.1,168.20,1106.46,1,0,0,1,0,1,1,0,2845.501260,51593.969849
38348,2019_38349,2019-09-07,관악구,신대방2동,봉천동,관악구_신대방2동,관악구_봉천동,37.491211,126.924990,1,4,7,32,39,99,0.038907,622.0,0.755516,5.0,0.000005,1127.0,0.342743,47.0,0.940026,8.0,84.04,11.0,1.0,0.0,0.0,1.0,0.0,23.0,1289.4,180.84,912.66,1,0,0,1,0,1,1,0,2647.176482,67296.482412
39577,2019_39578,2019-09-02,관악구,신대방2동,봉천동,관악구_신대방2동,관악구_봉천동,37.491211,126.924990,1,4,7,32,39,99,0.038907,622.0,0.755516,5.0,0.000005,1127.0,0.342743,47.0,0.940026,8.0,59.94,15.0,1.0,0.0,0.0,1.0,0.0,23.0,1209.6,168.99,1139.57,1,0,0,1,0,1,1,0,2956.846962,53612.864322
45033,2019_45034,2019-08-08,관악구,신대방2동,봉천동,관악구_신대방2동,관악구_봉천동,37.491211,126.924990,2,4,7,31,36,101,0.038907,622.0,0.755516,5.0,0.000005,1127.0,0.342743,47.0,0.940026,8.0,59.94,23.0,1.0,0.0,0.0,1.0,0.0,23.0,1214.5,171.59,1143.44,1,0,0,1,0,1,1,0,2845.501260,51593.969849
46612,2019_46613,2019-07-31,관악구,신대방2동,봉천동,관악구_신대방2동,관악구_봉천동,37.491211,126.924990,1,3,8,42,49,92,0.038907,622.0,0.755516,5.0,0.000005,1127.0,0.342743,47.0,0.940026,8.0,84.04,21.0,1.0,0.0,0.0,1.0,0.0,23.0,1182.0,171.57,1088.45,1,0,0,1,0,1,1,0,2470.698050,62810.050251


In [83]:
# 조건 생성
mask = final['gu_adong'] == '관악구_신대방2동'

# 조건에 맞는 행에서 열 값 변경
final.loc[mask, 'a_dong'] = final.loc[mask, 'a_dong'].replace('신대방2동', '보라매동')
final.loc[mask, 'l_dong'] = final.loc[mask, 'l_dong'].replace('신대방2동', '보라매동')
final.loc[mask, 'gu_adong'] = final.loc[mask, 'gu_adong'].replace('관악구_신대방2동', '관악구_보라매동')
final.loc[mask, 'gu_ldong'] = final.loc[mask, 'gu_ldong'].replace('관악구_신대방2동', '관악구_보라매동')


In [84]:
final[final['gu_adong']=='관악구_신대방2동']


,NO,contract_date,gu,a_dong,l_dong,gu_adong,gu_ldong,latitude,longitude,adong_prev_1_month,adong_prev_3_months,adong_prev_6_months,ldong_prev_1_month,ldong_prev_3_months,ldong_prev_6_months,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,USD_exchange_rate,CNY_exchange_rate,JPY_exchange_rate,K_progressive_president,K_conservative_president,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house,pyeong_price,adjusted_price


In [85]:
final[final['gu_adong']=='마포구_효창동']


,NO,contract_date,gu,a_dong,l_dong,gu_adong,gu_ldong,latitude,longitude,adong_prev_1_month,adong_prev_3_months,adong_prev_6_months,ldong_prev_1_month,ldong_prev_3_months,ldong_prev_6_months,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,USD_exchange_rate,CNY_exchange_rate,JPY_exchange_rate,K_progressive_president,K_conservative_president,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house,pyeong_price,adjusted_price
2106,2019_02107,2019-12-23,마포구,효창동,신공덕동,마포구_효창동,마포구_신공덕동,37.542281,126.958097,1,4,6,22,54,100,0.108876,781.0,0.386488,10.0,0.170027,961.0,0.102108,66.0,0.498383,23.0,121.38,2.0,1.0,0.0,0.0,0.0,1.0,32.0,1162.6,166.14,1061.98,1,0,0,1,0,1,1,0,2749.242605,100944.723618
16555,2019_16556,2019-11-13,마포구,효창동,신공덕동,마포구_효창동,마포구_신공덕동,37.542281,126.958097,2,5,5,31,60,126,0.108875,781.0,0.386486,10.0,0.170026,961.0,0.102106,66.0,0.498380,23.0,71.77,3.0,1.0,0.0,0.0,0.0,1.0,32.0,1163.5,166.18,1067.68,1,0,0,1,0,1,1,0,3358.057278,72904.522613
25627,2019_25628,2019-10-19,마포구,효창동,신공덕동,마포구_효창동,마포구_신공덕동,37.542281,126.958097,1,3,3,27,59,131,0.108876,781.0,0.386488,10.0,0.170027,961.0,0.102108,66.0,0.498383,23.0,60.40,6.0,1.0,0.0,0.0,0.0,1.0,32.0,1289.4,180.84,912.66,1,0,0,1,0,1,1,0,3621.869200,66174.874372
28813,2019_28814,2019-10-11,마포구,효창동,신공덕동,마포구_효창동,마포구_신공덕동,37.542281,126.958097,1,3,3,27,59,131,0.108876,781.0,0.386488,10.0,0.170027,961.0,0.102108,66.0,0.498383,23.0,60.40,5.0,1.0,0.0,0.0,0.0,1.0,32.0,1196.1,167.87,1108.48,1,0,0,1,0,1,1,0,3621.869200,66174.874372
79822,2020_04726,2020-12-12,마포구,효창동,신공덕동,마포구_효창동,마포구_신공덕동,37.542281,126.958097,1,3,7,19,51,71,0.108876,781.0,0.386488,10.0,0.170027,961.0,0.102108,66.0,0.498383,23.0,84.69,4.0,1.0,0.0,0.0,0.0,1.0,33.0,1289.4,180.84,912.66,1,0,0,1,0,1,1,0,4264.727915,109256.400000
82616,2020_07520,2020-12-01,마포구,효창동,신공덕동,마포구_효창동,마포구_신공덕동,37.542281,126.958097,1,3,7,19,51,71,0.108876,781.0,0.386488,10.0,0.170027,961.0,0.102108,66.0,0.498383,23.0,60.40,3.0,1.0,0.0,0.0,0.0,1.0,33.0,1104.9,168.03,1058.74,1,0,0,1,0,1,1,0,4825.373364,88164.000000
95652,2020_20556,2020-09-17,마포구,효창동,공덕동,마포구_효창동,마포구_공덕동,37.548725,126.962805,1,3,8,3,12,56,0.065401,538.0,0.502849,10.0,0.041191,623.0,0.178024,77.0,0.867931,16.0,95.25,7.0,1.0,0.0,0.0,1.0,0.0,16.0,1177.8,173.88,1121.66,1,0,0,1,0,1,1,0,3292.264441,94860.000000
107115,2020_32019,2020-07-15,마포구,효창동,신공덕동,마포구_효창동,마포구_신공덕동,37.542281,126.958097,1,4,7,28,57,78,0.108876,781.0,0.386488,10.0,0.170027,961.0,0.102108,66.0,0.498383,23.0,60.40,2.0,1.0,0.0,0.0,1.0,0.0,33.0,1206.4,172.10,1124.53,1,0,0,1,0,1,1,0,4581.050662,83700.000000
185188,2021_25957,2021-04-30,마포구,효창동,공덕동,마포구_효창동,마포구_공덕동,37.548675,126.962560,2,4,8,9,18,31,0.091291,539.0,0.481576,15.0,0.046712,664.0,0.187602,77.0,0.850879,16.0,84.81,2.0,1.0,0.0,1.0,0.0,0.0,20.0,1107.4,171.20,1016.90,1,0,1,0,1,0,1,0,2908.107791,74607.242340
209160,2022_06506,2022-05-15,마포구,효창동,공덕동,마포구_효창동,마포구_공덕동,37.548675,126.962560,1,4,7,2,11,21,0.091295,539.0,0.481573,15.0,0.046715,664.0,0.187605,77.0,0.850876,16.0,84.76,6.0,1.0,0.0,1.0,0.0,0.0,21.0,1289.4,180.84,912.66,0,1,1,0,1,0,1,0,3510.169891,90000.000000


In [86]:
# 조건 생성: gu_adong이 "마포구_효창동"인 행
mask = final['gu_adong'] == '마포구_효창동'

# 조건에 맞는 행에서 gu, gu_adong, gu_ldong 값 변경
final.loc[mask, 'gu'] = '용산구'
final.loc[mask, 'gu_adong'] = '용산구_효창동'
final.loc[mask, 'gu_ldong'] = '용산구_효창동'


In [87]:
final[final['gu_adong']=='용산구_효창동']

,NO,contract_date,gu,a_dong,l_dong,gu_adong,gu_ldong,latitude,longitude,adong_prev_1_month,adong_prev_3_months,adong_prev_6_months,ldong_prev_1_month,ldong_prev_3_months,ldong_prev_6_months,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,USD_exchange_rate,CNY_exchange_rate,JPY_exchange_rate,K_progressive_president,K_conservative_president,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house,pyeong_price,adjusted_price
2106,2019_02107,2019-12-23,용산구,효창동,신공덕동,용산구_효창동,용산구_효창동,37.542281,126.958097,1,4,6,22,54,100,0.108876,781.0,0.386488,10.0,0.170027,961.0,0.102108,66.0,0.498383,23.0,121.3800,2.0,1.0,0.0,0.0,0.0,1.0,32.0,1162.6,166.14,1061.98,1,0,0,1,0,1,1,0,2749.242605,100944.723618
3574,2019_03575,2019-12-16,용산구,효창동,효창동,용산구_효창동,용산구_효창동,37.540496,126.957796,4,20,31,4,20,31,0.075952,808.0,0.577804,5.0,0.072794,1021.0,0.139372,69.0,0.422924,23.0,84.4500,21.0,4.0,0.0,0.0,0.0,1.0,9.0,1171.7,168.14,1071.46,1,0,0,1,0,1,1,0,6585.811473,168241.206030
6570,2019_06571,2019-12-09,용산구,효창동,효창동,용산구_효창동,용산구_효창동,37.542317,126.963811,4,20,31,4,20,31,0.191576,632.0,0.539822,5.0,0.255868,631.0,0.252778,60.0,0.436402,26.0,160.3000,5.0,1.0,0.0,0.0,0.0,1.0,15.0,1189.7,168.88,1095.49,1,0,0,1,0,1,1,0,2266.784543,109917.587940
7685,2019_07686,2019-12-06,용산구,효창동,효창동,용산구_효창동,용산구_효창동,37.540831,126.957066,4,20,31,4,20,31,0.042332,802.0,0.584040,5.0,0.146908,1044.0,0.181651,48.0,0.511911,18.0,59.2800,14.0,1.0,0.0,0.0,0.0,1.0,13.0,1190.2,168.82,1094.49,1,0,0,1,0,1,1,0,5285.258133,94775.879397
8341,2019_08342,2019-12-04,용산구,효창동,효창동,용산구_효창동,용산구_효창동,37.540496,126.957796,4,20,31,4,20,31,0.075952,808.0,0.577804,5.0,0.072794,1021.0,0.139372,69.0,0.422924,23.0,59.4100,2.0,4.0,0.0,0.0,0.0,1.0,9.0,1186.2,168.56,1091.81,1,0,0,1,0,1,1,0,6428.288529,115525.628141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242871,2023_27418,2023-04-10,용산구,효창동,신공덕동,용산구_효창동,용산구_효창동,37.542281,126.958097,1,4,7,3,9,14,0.108876,781.0,0.386488,10.0,0.170027,961.0,0.102108,66.0,0.498383,23.0,60.4000,6.0,1.0,0.0,1.0,0.0,0.0,36.0,1318.9,191.61,999.36,0,1,1,0,1,0,0,1,4460.640728,81500.000000
243222,2023_27769,2023-04-07,용산구,효창동,효창동,용산구_효창동,용산구_효창동,37.540831,126.957066,2,4,8,2,4,8,0.042332,802.0,0.584040,5.0,0.146908,1044.0,0.181651,48.0,0.511911,18.0,59.2800,9.0,1.0,0.0,1.0,0.0,0.0,17.0,1318.7,191.50,999.77,0,1,1,0,1,0,0,1,4907.395412,88000.000000
244018,2023_28565,2023-03-31,용산구,효창동,효창동,용산구_효창동,용산구_효창동,37.539302,126.962948,1,4,8,1,4,8,0.059840,672.0,0.740062,5.0,0.141446,607.0,0.220613,51.0,0.174132,22.0,84.9204,12.0,4.0,1.0,0.0,0.0,0.0,4.0,1303.8,189.13,981.44,0,1,1,0,1,0,0,1,6501.012713,167000.000000
245272,2023_29819,2023-03-17,용산구,효창동,효창동,용산구_효창동,용산구_효창동,37.539302,126.962948,1,4,8,1,4,8,0.059840,672.0,0.740062,5.0,0.141446,607.0,0.220613,51.0,0.174132,22.0,84.9204,1.0,4.0,1.0,0.0,0.0,0.0,4.0,1312.1,190.10,983.40,0,1,1,0,1,0,0,1,5956.017635,153000.000000


In [88]:
final['contract_date'] = pd.to_datetime(final['contract_date'])
final['year'] = final['contract_date'].dt.year
final['year'] = final['year'].astype('int16')
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 47 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   NO                        251096 non-null  object        
 1   contract_date             251096 non-null  datetime64[ns]
 2   gu                        251096 non-null  object        
 3   a_dong                    251096 non-null  object        
 4   l_dong                    251096 non-null  object        
 5   gu_adong                  251096 non-null  object        
 6   gu_ldong                  251096 non-null  object        
 7   latitude                  251096 non-null  float64       
 8   longitude                 251096 non-null  float64       
 9   adong_prev_1_month        251096 non-null  int64         
 10  adong_prev_3_months       251096 non-null  int64         
 11  adong_prev_6_months       251096 non-null  int64         
 12  ld

In [89]:
pop_density_2019.columns = ['gu_adong', 'gu', 'a_dong',  'population_density', 'year']
pop_density_2020.columns = ['gu_adong', 'gu', 'a_dong', 'population_density', 'year']
pop_density_2021.columns = ['gu_adong', 'gu', 'a_dong',  'population_density', 'year']
pop_density_2022.columns = ['gu_adong', 'gu', 'a_dong',  'population_density', 'year']
pop_density_2023.columns = ['gu_adong', 'gu', 'a_dong',  'population_density', 'year']


In [90]:
all_pop_density_2019_2023 = pd.concat([pop_density_2019, pop_density_2020, pop_density_2021, pop_density_2022, pop_density_2023], ignore_index=True)
all_pop_density_2019_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145 entries, 0 to 2144
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gu_adong            2145 non-null   object 
 1   gu                  2145 non-null   object 
 2   a_dong              2145 non-null   object 
 3   population_density  2145 non-null   float64
 4   year                2145 non-null   int16  
dtypes: float64(1), int16(1), object(3)
memory usage: 71.3+ KB


In [91]:
final_copy = pd.merge(
    final,
    all_pop_density_2019_2023[['gu_adong', 'year',  'population_density']], 
    how='outer', on=['gu_adong', 'year']
)
final_copy

,NO,contract_date,gu,a_dong,l_dong,gu_adong,gu_ldong,latitude,longitude,adong_prev_1_month,adong_prev_3_months,adong_prev_6_months,ldong_prev_1_month,ldong_prev_3_months,ldong_prev_6_months,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,USD_exchange_rate,CNY_exchange_rate,JPY_exchange_rate,K_progressive_president,K_conservative_president,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house,pyeong_price,adjusted_price,year,population_density
0,2019_00620,2019-12-29,강남구,개포1동,개포동,강남구_개포1동,강남구_개포동,37.480287,127.057848,16.0,57.0,141.0,6.0,44.0,91.0,0.240778,325.0,1.326975,1.0,0.324330,444.0,0.237504,72.0,0.737155,9.0,50.38,1.0,1.0,0.0,0.0,0.0,1.0,37.0,1289.4,180.84,912.66,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,18767.209689,286010.050251,2019,5449.0
1,2019_02480,2019-12-21,강남구,개포1동,개포동,강남구_개포1동,강남구_개포동,37.480287,127.057848,16.0,57.0,141.0,6.0,44.0,91.0,0.240778,325.0,1.326975,1.0,0.324330,444.0,0.237504,72.0,0.737155,9.0,35.87,5.0,1.0,0.0,0.0,0.0,1.0,37.0,1289.4,180.84,912.66,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,25842.011843,280402.010050,2019,5449.0
2,2019_03315,2019-12-18,강남구,개포1동,개포동,강남구_개포1동,강남구_개포동,37.485406,127.054710,16.0,57.0,141.0,6.0,44.0,91.0,0.285093,350.0,1.206379,1.0,0.000000,828.0,0.219089,79.0,0.487774,28.0,182.20,1.0,1.0,0.0,0.0,0.0,1.0,35.0,1166.7,166.78,1065.33,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,5291.059733,291618.090452,2019,5449.0
3,2019_03829,2019-12-16,강남구,개포1동,개포동,강남구_개포1동,강남구_개포동,37.480287,127.057848,16.0,57.0,141.0,6.0,44.0,91.0,0.240778,325.0,1.326975,1.0,0.324330,444.0,0.237504,72.0,0.737155,9.0,41.98,3.0,1.0,0.0,0.0,0.0,1.0,37.0,1171.7,168.14,1071.46,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,16428.132583,208619.095477,2019,5449.0
4,2019_04623,2019-12-14,강남구,개포1동,개포동,강남구_개포1동,강남구_개포동,37.483280,127.054393,16.0,57.0,141.0,6.0,44.0,91.0,0.149098,389.0,1.301747,1.0,0.000000,621.0,0.372332,87.0,0.634700,22.0,95.40,13.0,1.0,0.0,0.0,0.0,1.0,35.0,1289.4,180.84,912.66,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,7598.293695,219274.371859,2019,5449.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253521,2023_25864,2023-04-24,중랑구,중화2동,중화동,중랑구_중화2동,중랑구_중화동,37.594532,127.075918,1.0,4.0,7.0,5.0,11.0,16.0,0.059521,506.0,1.973710,0.0,0.096594,875.0,0.543370,37.0,0.048189,20.0,17.94,9.0,1.0,0.0,1.0,0.0,0.0,11.0,1327.9,192.89,990.86,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2137.529543,11600.000000,2023,24085.0
253522,2023_29540,2023-03-20,중랑구,중화2동,중화동,중랑구_중화2동,중랑구_중화동,37.594532,127.075918,1.0,4.0,7.0,5.0,8.0,13.0,0.059521,506.0,1.973710,0.0,0.096594,875.0,0.543370,37.0,0.048189,20.0,17.94,9.0,1.0,1.0,0.0,0.0,0.0,11.0,1303.6,189.26,987.09,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2119.102564,11500.000000,2023,24085.0
253523,2023_31678,2023-02-28,중랑구,중화2동,중화동,중랑구_중화2동,중랑구_중화동,37.604031,127.078475,2.0,4.0,8.0,2.0,6.0,9.0,0.133004,431.0,1.710154,0.0,0.060225,821.0,0.272001,63.0,0.187117,19.0,84.78,4.0,1.0,1.0,0.0,0.0,0.0,19.0,1317.4,188.79,967.36,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1949.634348,50000.000000,2023,24085.0
253524,2023_34294,2023-01-30,중랑구,중화2동,중화동,중랑구_중화2동,중랑구_중화동,37.594532,127.075918,1.0,3.0,7.0,2.0,4.0,9.0,0.059521,506.0,1.973710,0.0,0.096594,875.0,0.543370,37.0,0.048189,20.0,17.94,12.0,1.0,1.0,0.0,0.0,0.0,11.0,1230.2,182.33,947.51,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2128.316054,11550.000000,2023,24085.0


In [92]:
final_copy = final_copy.drop_duplicates(subset='NO')

In [93]:
final_copy.isna().sum()

NO                          1
contract_date               1
gu                          1
a_dong                      1
l_dong                      1
gu_adong                    0
gu_ldong                    1
latitude                    1
longitude                   1
adong_prev_1_month          1
adong_prev_3_months         1
adong_prev_6_months         1
ldong_prev_1_month          1
ldong_prev_3_months         1
ldong_prev_6_months         1
closest_bus_stop_dist_km    1
bus_stop_score              1
closest_park_dist_km        1
park_score                  1
closest_hospital_dist_km    1
hospital_score              1
closest_school_dist_km      1
school_score                1
closest_station_dist_km     1
station_score               1
exclusive_area_m2           1
floor                       1
brand_label_encoded         1
Q_1                         1
Q_2                         1
Q_3                         1
Q_4                         1
apt_age                     1
USD_exchan

In [94]:
final_copy.describe()

,contract_date,latitude,longitude,adong_prev_1_month,adong_prev_3_months,adong_prev_6_months,ldong_prev_1_month,ldong_prev_3_months,ldong_prev_6_months,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,USD_exchange_rate,CNY_exchange_rate,JPY_exchange_rate,K_progressive_president,K_conservative_president,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house,pyeong_price,adjusted_price,year,population_density
count,251096,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.00000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,251096.000000,2.510960e+05,251097.000000,251097.000000
mean,2020-12-03 10:50:45.439194624,37.555110,126.995693,21.238120,58.967323,114.593904,18.210641,54.440330,106.789148,0.128200,476.482481,1.230374,3.951664,0.143288,896.862873,0.298340,58.125960,0.643731,14.949489,75.496983,9.280522,1.611189,0.197494,0.272661,0.271076,0.25877,19.064613,1223.858578,176.008142,1012.238997,0.830666,0.169334,0.351595,0.648405,0.351595,0.648405,0.857875,0.142125,4230.544008,9.764059e+04,2020.401638,24919.916112
min,2019-01-01 00:00:00,37.434420,126.806400,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.004775,59.000000,0.060915,0.000000,0.000000,4.000000,0.000000,1.000000,0.023343,0.000000,10.020000,-3.000000,1.000000,0.000000,0.000000,0.000000,0.00000,-1.000000,1082.700000,162.760000,860.660000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,526.948275,6.056683e+03,2019.000000,285.000000
25%,2019-11-24 00:00:00,37.506884,126.917908,6.000000,21.000000,43.000000,4.000000,20.000000,43.000000,0.068289,366.000000,0.712165,0.000000,0.032370,546.000000,0.184990,43.000000,0.362480,7.000000,59.420000,4.000000,1.000000,0.000000,0.000000,0.000000,0.00000,12.000000,1172.700000,170.030000,912.660000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2779.866085,5.439799e+04,2019.000000,15746.000000
50%,2020-07-10 00:00:00,37.549193,127.017588,14.000000,43.000000,87.000000,11.000000,39.000000,81.000000,0.113255,470.000000,1.137639,1.000000,0.118323,801.000000,0.274222,56.000000,0.550739,13.000000,78.190000,8.000000,1.000000,0.000000,0.000000,0.000000,0.00000,19.000000,1210.800000,175.060000,1034.440000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,3703.430353,8.019497e+04,2020.000000,24523.000000
75%,2021-07-16 00:00:00,37.599466,127.065709,29.000000,82.000000,162.000000,24.000000,72.000000,144.000000,0.169379,580.000000,1.629728,6.000000,0.213041,1088.000000,0.388203,72.000000,0.782755,21.000000,84.960000,13.000000,1.000000,0.000000,1.000000,1.000000,1.00000,26.000000,1289.400000,180.840000,1086.360000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,5078.947016,1.195000e+05,2021.000000,32947.000000
max,2023-12-31 00:00:00,37.687503,127.179998,259.000000,356.000000,617.000000,259.000000,416.000000,587.000000,0.705235,1120.000000,3.488772,30.000000,1.357089,7162.000000,1.297277,141.000000,4.269431,84.000000,317.360000,68.000000,4.000000,1.000000,1.000000,1.000000,1.00000,62.000000,1436.600000,201.790000,1172.730000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,29129.459395,1.800000e+06,2023.000000,58667.000000
std,NaN,0.056496,0.090399,22.278243,52.031174,94.649306,21.123235,49.298707,88.206710,0.081155,162.206958,0.658943,4.803361

In [95]:
final_copy.isna().sum()

NO                          1
contract_date               1
gu                          1
a_dong                      1
l_dong                      1
gu_adong                    0
gu_ldong                    1
latitude                    1
longitude                   1
adong_prev_1_month          1
adong_prev_3_months         1
adong_prev_6_months         1
ldong_prev_1_month          1
ldong_prev_3_months         1
ldong_prev_6_months         1
closest_bus_stop_dist_km    1
bus_stop_score              1
closest_park_dist_km        1
park_score                  1
closest_hospital_dist_km    1
hospital_score              1
closest_school_dist_km      1
school_score                1
closest_station_dist_km     1
station_score               1
exclusive_area_m2           1
floor                       1
brand_label_encoded         1
Q_1                         1
Q_2                         1
Q_3                         1
Q_4                         1
apt_age                     1
USD_exchan

In [96]:
final_copy.to_csv('Data_Preprocessing/merged_aptinfo_distance_politics_transactionCount_E_pop_density_df.csv', index=False, encoding='utf-8')

In [204]:
final_copy

,NO,contract_date,gu,a_dong,l_dong,gu_adong,gu_ldong,latitude,longitude,adong_prev_1_month,adong_prev_3_months,adong_prev_6_months,ldong_prev_1_month,ldong_prev_3_months,ldong_prev_6_months,closest_bus_stop_dist_km,bus_stop_score,closest_park_dist_km,park_score,closest_hospital_dist_km,hospital_score,closest_school_dist_km,school_score,closest_station_dist_km,station_score,exclusive_area_m2,floor,brand_label_encoded,Q_1,Q_2,Q_3,Q_4,apt_age,USD_exchange_rate,CNY_exchange_rate,JPY_exchange_rate,K_progressive_president,K_conservative_president,A_progressive_president,A_conservative_president,A_progressive_senate,A_conservative_senate,A_progressive_house,A_conservative_house,pyeong_price,adjusted_price,year,population_density
0,2019_00620,2019-12-29,강남구,개포1동,개포동,강남구_개포1동,강남구_개포동,37.480287,127.057848,16.0,57.0,141.0,6.0,44.0,91.0,0.240778,325.0,1.326975,1.0,0.324330,444.0,0.237504,72.0,0.737155,9.0,50.38,1.0,1.0,0.0,0.0,0.0,1.0,37.0,1289.4,180.84,912.66,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,18767.209689,286010.050251,2019,5449.0
1,2019_02480,2019-12-21,강남구,개포1동,개포동,강남구_개포1동,강남구_개포동,37.480287,127.057848,16.0,57.0,141.0,6.0,44.0,91.0,0.240778,325.0,1.326975,1.0,0.324330,444.0,0.237504,72.0,0.737155,9.0,35.87,5.0,1.0,0.0,0.0,0.0,1.0,37.0,1289.4,180.84,912.66,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,25842.011843,280402.010050,2019,5449.0
2,2019_03315,2019-12-18,강남구,개포1동,개포동,강남구_개포1동,강남구_개포동,37.485406,127.054710,16.0,57.0,141.0,6.0,44.0,91.0,0.285093,350.0,1.206379,1.0,0.000000,828.0,0.219089,79.0,0.487774,28.0,182.20,1.0,1.0,0.0,0.0,0.0,1.0,35.0,1166.7,166.78,1065.33,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,5291.059733,291618.090452,2019,5449.0
3,2019_03829,2019-12-16,강남구,개포1동,개포동,강남구_개포1동,강남구_개포동,37.480287,127.057848,16.0,57.0,141.0,6.0,44.0,91.0,0.240778,325.0,1.326975,1.0,0.324330,444.0,0.237504,72.0,0.737155,9.0,41.98,3.0,1.0,0.0,0.0,0.0,1.0,37.0,1171.7,168.14,1071.46,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,16428.132583,208619.095477,2019,5449.0
4,2019_04623,2019-12-14,강남구,개포1동,개포동,강남구_개포1동,강남구_개포동,37.483280,127.054393,16.0,57.0,141.0,6.0,44.0,91.0,0.149098,389.0,1.301747,1.0,0.000000,621.0,0.372332,87.0,0.634700,22.0,95.40,13.0,1.0,0.0,0.0,0.0,1.0,35.0,1289.4,180.84,912.66,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,7598.293695,219274.371859,2019,5449.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253521,2023_25864,2023-04-24,중랑구,중화2동,중화동,중랑구_중화2동,중랑구_중화동,37.594532,127.075918,1.0,4.0,7.0,5.0,11.0,16.0,0.059521,506.0,1.973710,0.0,0.096594,875.0,0.543370,37.0,0.048189,20.0,17.94,9.0,1.0,0.0,1.0,0.0,0.0,11.0,1327.9,192.89,990.86,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2137.529543,11600.000000,2023,24085.0
253522,2023_29540,2023-03-20,중랑구,중화2동,중화동,중랑구_중화2동,중랑구_중화동,37.594532,127.075918,1.0,4.0,7.0,5.0,8.0,13.0,0.059521,506.0,1.973710,0.0,0.096594,875.0,0.543370,37.0,0.048189,20.0,17.94,9.0,1.0,1.0,0.0,0.0,0.0,11.0,1303.6,189.26,987.09,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2119.102564,11500.000000,2023,24085.0
253523,2023_31678,2023-02-28,중랑구,중화2동,중화동,중랑구_중화2동,중랑구_중화동,37.604031,127.078475,2.0,4.0,8.0,2.0,6.0,9.0,0.133004,431.0,1.710154,0.0,0.060225,821.0,0.272001,63.0,0.187117,19.0,84.78,4.0,1.0,1.0,0.0,0.0,0.0,19.0,1317.4,188.79,967.36,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1949.634348,50000.000000,2023,24085.0
253524,2023_34294,2023-01-30,중랑구,중화2동,중화동,중랑구_중화2동,중랑구_중화동,37.594532,127.075918,1.0,3.0,7.0,2.0,4.0,9.0,0.059521,506.0,1.973710,0.0,0.096594,875.0,0.543370,37.0,0.048189,20.0,17.94,12.0,1.0,1.0,0.0,0.0,0.0,11.0,1230.2,182.33,947.51,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,2128.316054,11550.000000,2023,24085.0


In [ ]:
selected_final = final_copy.drop(columns = ['NO','contract_date','gu','a_dong','l_dong','gu_adong','gu_ldong'])

In [ ]:
selected_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253502 entries, 0 to 253525
Data columns (total 41 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   latitude                  253502 non-null  float64
 1   longitude                 253502 non-null  float64
 2   adong_prev_1_month        253502 non-null  float64
 3   adong_prev_3_months       253502 non-null  float64
 4   adong_prev_6_months       253502 non-null  float64
 5   ldong_prev_1_month        253502 non-null  float64
 6   ldong_prev_3_months       253502 non-null  float64
 7   ldong_prev_6_months       253502 non-null  float64
 8   closest_bus_stop_dist_km  253502 non-null  float64
 9   bus_stop_score            253502 non-null  float64
 10  closest_park_dist_km      253502 non-null  float64
 11  park_score                253502 non-null  float64
 12  closest_hospital_dist_km  253502 non-null  float64
 13  hospital_score            253502 non-null  float6

In [ ]:
sample_data = selected_final.sample(n=15000, random_state = 42)

# sample_data['population_density'] = sample_data['population_density'].astype('int')
sample_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 162301 to 25723
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   latitude                  15000 non-null  float64
 1   longitude                 15000 non-null  float64
 2   adong_prev_1_month        15000 non-null  float64
 3   adong_prev_3_months       15000 non-null  float64
 4   adong_prev_6_months       15000 non-null  float64
 5   ldong_prev_1_month        15000 non-null  float64
 6   ldong_prev_3_months       15000 non-null  float64
 7   ldong_prev_6_months       15000 non-null  float64
 8   closest_bus_stop_dist_km  15000 non-null  float64
 9   bus_stop_score            15000 non-null  float64
 10  closest_park_dist_km      15000 non-null  float64
 11  park_score                15000 non-null  float64
 12  closest_hospital_dist_km  15000 non-null  float64
 13  hospital_score            15000 non-null  float64
 14  closes

# 모델링

In [642]:
from sklearn.model_selection import train_test_split

X = sample_data.drop(columns = ['K_conservative_president','A_conservative_president','A_conservative_senate','A_conservative_house','pyeong_price','adjusted_price'])
y = sample_data['adjusted_price']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25, random_state= 42)

from sklearn.ensemble import RandomForestRegressor
# 정당컬럼 포함
# n_estimators 지정 안할시 0.985, 0.909
# n_estimators = 200 일시 0.986, 0.910

# 정당 컬럼 진보만 포함
# n_estimators = 100일시 0.985, 0.909
# n_estimators = 200일시 0.986. 0.910
rf = RandomForestRegressor(n_estimators = 200, random_state =42,max_depth =  None, min_samples_leaf = 1, min_samples_split =  2)
rf.fit(X_train, y_train)

rf.score(X_train,y_train) , rf.score(X_test,y_test)

(0.9863313371729822, 0.9100616678219977)

In [643]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error

mae = mean_absolute_error(y_test,rf.predict(X_test))
mse = mean_squared_error(y_test, rf.predict(X_test))
rmse = mse**0.5

print(mae, mse, rmse)
print(1 - mae/np.mean(y_test))

12496.667519127743 443224536.023807 21052.898518346756
0.87481205830919


In [644]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf =RandomForestRegressor(random_state=42)

# 그리드 서치 실행
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='r2',
    cv=3,  # 3개의 폴드를 사용
    verbose=2,
    n_jobs=-1
)

# 그리드 서치 실행
grid_search.fit(X_train, y_train)



Fitting 3 folds for each of 81 candidates, totalling 243 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='r2', verbose=2)

In [645]:
print(grid_search.best_params_)

{'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [646]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# 데이터 분할
X = sample_data.drop(columns = ['K_conservative_president','A_conservative_president','A_conservative_senate','A_conservative_house','pyeong_price','adjusted_price'])
y = sample_data['adjusted_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

# XGBoost 모델 초기화
xgb = XGBRegressor(n_estimators=100, random_state=42)

# 모델 학습
xgb.fit(X_train, y_train)

# 학습 데이터와 테스트 데이터에 대한 R^2 스코어 출력
xgb.score(X_train, y_train), xgb.score(X_test, y_test)


(0.9875757563023707, 0.9262001449043405)

In [647]:
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor

# 데이터 분할
X = sample_data.drop(columns = ['K_conservative_president','A_conservative_president','A_conservative_senate','A_conservative_house','pyeong_price','adjusted_price'])
y = sample_data['adjusted_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

# XGBoost 모델 초기화
xgb = XGBRegressor(random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='r2',   # R-squared 기준으로 평가
    cv=3,           # 교차 검증 폴드 수
    verbose=2,      # 진행 상황 출력
    n_jobs=-1       # 병렬 처리
)

# 그리드 서치 실행
grid_search.fit(X_train, y_train)

# 최적의 파라미터 출력
print("Best parameters:", grid_search.best_params_)

# 최적 모델로 평가
print("Train R^2:", grid_search.score(X_train, y_train))
print("Test R^2:", grid_search.score(X_test, y_test))


Fitting 3 folds for each of 243 candidates, totalling 729 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'subsample': 0.9}
Train R^2: 0.9959418902096262
Test R^2: 0.9365701810979395


In [648]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

# 데이터 분할
X = sample_data.drop(columns = ['K_conservative_president','A_conservative_president','A_conservative_senate','A_conservative_house','pyeong_price','adjusted_price'])
y = sample_data['adjusted_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

# XGBoost 모델 초기화
xgb = XGBRegressor(n_estimators=100, random_state=42)

# 교차 검증 수행
cv_scores = cross_val_score(xgb, X_train, y_train, cv=3, scoring='r2', n_jobs=-1)

# 교차 검증 결과 출력
print("Cross-validation R^2 scores:", cv_scores)
print("Mean R^2:", cv_scores.mean())
print("Standard deviation of R^2:", cv_scores.std())


Cross-validation R^2 scores: [0.9028379  0.89953264 0.89699124]
Mean R^2: 0.8997872584566767
Standard deviation of R^2: 0.0023936688977527873


In [649]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd

# 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 모델 초기화
rf = RandomForestRegressor(random_state=42)

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='r2',  # R-squared 기준으로 평가
    cv=3,          # 교차 검증 폴드 수
    verbose=2,      # 진행 상황 출력
    n_jobs=-1       # 병렬 처리
)

# 그리드 서치 실행
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='r2', verbose=2)

In [650]:
# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid_search.best_params_)

# 최적의 점수 출력
print("Best Score:", grid_search.best_score_)

# 테스트 세트 평가
best_model = grid_search.best_estimator_
print("Test Set Score:", best_model.score(X_test, y_test))


Best Parameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Score: 0.8846793612271489
Test Set Score: 0.9105623045639534


In [651]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

# X와 y는 이미 정의된 feature와 target입니다.
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# 교차검증: 5개의 폴드를 사용하여 성능 평가
cv_scores = cross_val_score(rf, X, y, cv=5, scoring='r2')  # 'r2'는 결정계수를 사용한 성능 평가

print(f"교차 검증 결과: {cv_scores}")
print(f"교차 검증 평균 R^2 점수: {cv_scores.mean()}")


교차 검증 결과: [0.90722539 0.89816978 0.91148332 0.91864243 0.90121931]
교차 검증 평균 R^2 점수: 0.9073480464213921
